**Navigation** : [← Lean-8-Agentic-Proving](Lean-8-Agentic-Proving.ipynb) | [Index](Lean-1-Setup.ipynb) | [Lean-10-LeanDojo →](Lean-10-LeanDojo.ipynb)

---



# Lean 9 : Multi-Agents avec Semantic Kernel

## 🎯 Architecture du Système Multi-Agents

### Vue d'ensemble

Notre système utilise **5 agents spécialisés** qui collaborent pour prouver des théorèmes Lean :

1. **SearchAgent** : Recherche de lemmes pertinents dans Mathlib
2. **TacticAgent** : Génération de tactiques Lean appropriées
3. **VerifierAgent** : Vérification formelle des preuves
4. **CriticAgent** : Analyse et suggestions d'amélioration
5. **CoordinatorAgent** : Orchestration et décisions stratégiques

### Pourquoi 5 agents ?

Chaque agent a une **responsabilité unique** (principe de séparation des préoccupations) :

- **Séparation des compétences** : Recherche ≠ Génération ≠ Vérification
- **Spécialisation** : Chaque LLM est prompté pour une tâche précise
- **Robustesse** : Si un agent échoue, les autres continuent
- **Traçabilité** : On sait quel agent a pris quelle décision

### Communication : État partagé vs Message passing

Deux approches classiques en multi-agents :

| **Message Passing** | **État Partagé** (notre choix) |
|---------------------|--------------------------------|
| Agents s'envoient des messages | Tous les agents lisent/écrivent un état central |
| Décentralisé | Centralisé |
| Complexe à orchestrer | Facile à suivre |
| Pas de snapshot global | Snapshot complet à chaque itération |

**Pourquoi état partagé ?**

- Besoin de **cohérence globale** (historique des tactiques, métriques)
- **Debugging facilité** : On peut inspecter l'état après chaque tour
- **Snapshots JSON** : Permet de reproduire exactement une session
- Semantic Kernel supporte ce pattern avec les **plugins**

## 1. Introduction : Semantic Kernel pour Preuves (Python)

### 5.1 Vue d'ensemble

Microsoft **Semantic Kernel** est un SDK qui permet d'orchestrer des LLMs avec des plugins, de la memoire et des agents intelligents. Nous allons implementer un systeme multi-agents pour theorem proving inspire des patterns utilises dans l'analyse argumentative (voir `Argument_Analysis` notebooks).

**Composants cles** :
- **Kernel** : Point d'entree principal, configure les services LLM
- **Plugins** : Fonctions appelables par les agents (decorated avec `@kernel_function`)
- **Agents** : Entites autonomes avec instructions et capacites
- **Orchestration** : Strategies de selection et terminaison des agents

### 5.2 Dependances

```python
# Installation
pip install semantic-kernel openai python-dotenv
```

## 📊 État Partagé : La Classe `ProofState`

La classe `ProofState` est le **cœur du système**. Elle contient :

### 1. Phase de preuve (`ProofPhase` enum)
```
INIT → SEARCH → TACTIC_GEN → VERIFICATION → REFINEMENT → COMPLETE
```

Chaque phase détermine **quel agent agit** :
- `INIT` → CoordinatorAgent décide de la stratégie
- `SEARCH` → SearchAgent cherche des lemmes
- `TACTIC_GEN` → TacticAgent génère une tactique
- `VERIFICATION` → VerifierAgent teste la preuve
- `REFINEMENT` → CriticAgent analyse et ajuste
- `COMPLETE` → Session terminée

### 2. Stratégie de preuve (`ProofStrategy` enum)

```python
EXPLORATION   # Recherche large de lemmes
REFINEMENT    # Ajustement d'une preuve existante
VALIDATION    # Vérification formelle
RECOVERY      # Récupération après erreur
```

La stratégie influence **quels lemmes rechercher** et **quelles tactiques essayer**.

### 3. Historique et métriques

- `tactic_history` : Liste de toutes les tactiques essayées (succès + échecs)
- `verification_results` : Résultats des vérifications Lean
- `current_proof` : Preuve en construction
- `error_count` : Nombre d'erreurs rencontrées

### 4. Snapshots JSON

À chaque itération, on peut sauvegarder l'état complet en JSON :

```json
{
  "phase": "TACTIC_GEN",
  "strategy": "EXPLORATION",
  "iteration": 5,
  "current_goal": "n + 0 = n",
  "tactic_history": [...],
  "current_proof": ["intro n", "rw [Nat.add_zero]"]
}
```

**Utilité** : Debugging, reproduction de bugs, benchmarking.

In [16]:
# =============================================================================
# Section 8.1 - ProofState: Etat Partage pour Multi-Agents
# =============================================================================
# Pattern inspire de RhetoricalAnalysisState dans Argument_Analysis
# Permet la synchronisation entre agents avec designation explicite

import os
import sys
import json
import time
import uuid
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Any, Tuple
from datetime import datetime
from enum import Enum
from pathlib import Path

# --- Detection robuste du repertoire du notebook ---
# Fonctionne sous Windows, Linux, et WSL
notebook_dir = None

# Chemins connus (Windows et WSL)
KNOWN_PATHS = [
    Path("/mnt/d/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean"),  # WSL
    Path("/mnt/c/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean"),  # WSL (C:)
    Path("d:/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean"),      # Windows
    Path("D:/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean"),      # Windows
]

# Strategie 1: Variable d'environnement LEAN_NOTEBOOK_DIR
if os.getenv("LEAN_NOTEBOOK_DIR"):
    notebook_dir = Path(os.getenv("LEAN_NOTEBOOK_DIR"))
    if not (notebook_dir / "lean_runner.py").exists():
        notebook_dir = None

# Strategie 2: Chemins connus
if not notebook_dir:
    for known_path in KNOWN_PATHS:
        if known_path.exists() and (known_path / "lean_runner.py").exists():
            notebook_dir = known_path
            break

# Strategie 3: Chercher dans cwd et parents
if not notebook_dir:
    cwd = Path.cwd()
    candidates = [cwd, cwd / "MyIA.AI.Notebooks" / "SymbolicAI" / "Lean"]

    # Remonter jusqu'a 5 niveaux
    current = cwd
    for _ in range(5):
        candidates.append(current)
        lean_path = current / "MyIA.AI.Notebooks" / "SymbolicAI" / "Lean"
        if lean_path.exists():
            candidates.append(lean_path)
        if current.parent == current:
            break
        current = current.parent

    for candidate in candidates:
        if candidate.exists() and (candidate / "lean_runner.py").exists():
            notebook_dir = candidate
            break

# Strategie 4: Fallback sur cwd
if not notebook_dir:
    notebook_dir = Path.cwd()
    print(f"[WARN] lean_runner.py non trouve, fallback sur: {notebook_dir}")

# --- Charger .env ---
try:
    from dotenv import load_dotenv
    env_paths = [
        notebook_dir / ".env",
        notebook_dir.parent / ".env",
        Path.home() / ".env"
    ]
    for p in env_paths:
        if p.exists():
            load_dotenv(p, override=True)
            print(f"Configuration chargee depuis: {p}")
            break
    else:
        print("Aucun fichier .env trouve")
except ImportError:
    print("python-dotenv non installe")

# --- Importer lean_runner.py ---
if notebook_dir and str(notebook_dir) not in sys.path:
    sys.path.insert(0, str(notebook_dir))

try:
    from lean_runner import LeanRunner, LeanResult
    print(f"lean_runner importe avec succes depuis {notebook_dir}")
except ImportError as e:
    print(f"ERREUR: Impossible d'importer lean_runner: {e}")
    print(f"Repertoire de travail: {Path.cwd()}")
    print(f"notebook_dir detecte: {notebook_dir}")
    print(f"sys.path: {sys.path[:5]}")
    raise

# --- Enumerations ---

class ProofStrategy(Enum):
    """Strategie de preuve en cours."""
    EXPLORATION = "exploration"      # Recherche initiale de lemmes
    REFINEMENT = "refinement"        # Affinage des tactiques
    VALIDATION = "validation"        # Verification finale
    RECOVERY = "recovery"            # Recuperation apres echecs

class TacticDifficulty(Enum):
    """Niveau de difficulte des tactiques."""
    SIMPLE = "simple"      # rfl, exact, omega
    INTERMEDIATE = "intermediate"  # simp, ring, linarith
    ADVANCED = "advanced"  # induction, cases

class ProofPhase(Enum):
    """Phase de la boucle de preuve."""
    INIT = "init"
    SEARCH = "search"
    GENERATE = "generate"
    VERIFY = "verify"
    ANALYZE = "analyze"
    COMPLETE = "complete"
    FAILED = "failed"

# --- ProofState: Etat partage entre agents ---

@dataclass
class TacticAttempt:
    """Une tentative de tactique."""
    tactic: str
    success: bool
    error: Optional[str] = None
    timestamp: datetime = field(default_factory=datetime.now)
    state_before: Optional[str] = None
    confidence: Optional[float] = None
    explanation: Optional[str] = None

@dataclass
class ProofState:
    """
    Etat partage entre les agents pour la preuve d'un theoreme.
    Permet la coordination sans couplage fort.
    """
    # Identifiants
    session_id: str = field(default_factory=lambda: str(uuid.uuid4())[:8])
    theorem_name: str = ""
    theorem_statement: str = ""

    # Etat de la preuve
    current_goal: str = ""
    current_proof: List[str] = field(default_factory=list)
    phase: ProofPhase = ProofPhase.INIT
    strategy: ProofStrategy = ProofStrategy.EXPLORATION

    # Resultats des agents
    discovered_lemmas: List[str] = field(default_factory=list)
    generated_tactics: List[str] = field(default_factory=list)
    tactic_history: List[TacticAttempt] = field(default_factory=list)

    # Metriques
    iteration: int = 0
    max_iterations: int = 10
    start_time: datetime = field(default_factory=datetime.now)

    # Erreurs et diagnostics
    last_error: Optional[str] = None
    final_proof: Optional[str] = None
    error_count: int = 0

    # Verification tracking
    verification_results: List[Dict[str, Any]] = field(default_factory=list)
    total_lean_time_ms: float = 0.0

    # Agent designation for orchestration
    _next_agent: Optional[str] = field(default=None, repr=False)

    def add_tactic_attempt(self, tactic: str, state_before: Optional[str] = None,
                           confidence: Optional[float] = None, explanation: Optional[str] = None,
                           success: bool = False, error: Optional[str] = None) -> str:
        """Enregistre une tentative de tactique."""
        attempt_id = f"attempt_{len(self.tactic_history) + 1}"
        self.tactic_history.append(TacticAttempt(
            tactic=tactic,
            success=success,
            error=error,
            state_before=state_before,
            confidence=confidence,
            explanation=explanation
        ))
        if success:
            self.current_proof.append(tactic)
        else:
            self.error_count += 1
            self.last_error = error
        return attempt_id

    def add_lemma(self, name: str, statement: str, namespace: str = "", relevance: float = 0.5) -> str:
        """Ajoute un lemme decouvert a la liste."""
        lemma_id = f"{namespace}.{name}" if namespace else name
        lemma_info = f"{lemma_id}: {statement} (relevance: {relevance})"
        if lemma_info not in self.discovered_lemmas:
            self.discovered_lemmas.append(lemma_info)
        return lemma_id

    def get_context_summary(self) -> str:
        """Resume le contexte pour les agents."""
        return f"""
Theoreme: {self.theorem_name}
Enonce: {self.theorem_statement}
But actuel: {self.current_goal}
Phase: {self.phase.value}
Strategie: {self.strategy.value}
Iteration: {self.iteration}/{self.max_iterations}
Tactiques reussies: {len(self.current_proof)}
Erreurs: {self.error_count}
Derniere erreur: {self.last_error or 'Aucune'}
""".strip()


    

    # --- Properties for compatibility ---
    @property
    def tactics_history(self) -> List[TacticAttempt]:
        """Alias pour tactic_history (compatibilite)."""
        return self.tactic_history

    @property
    def proof_complete(self) -> bool:
        """True si la preuve est complete."""
        return self.phase == ProofPhase.COMPLETE
    
    @proof_complete.setter
    def proof_complete(self, value: bool):
        """Definit la completion de la preuve."""
        if value:
            self.phase = ProofPhase.COMPLETE
        elif self.phase == ProofPhase.COMPLETE:
            self.phase = ProofPhase.VERIFY
    
    @property
    def iteration_count(self) -> int:
        """Alias pour iteration (compatibilite)."""
        return self.iteration
    
    @iteration_count.setter
    def iteration_count(self, value: int):
        """Definit le compteur d'iterations."""
        self.iteration = value

    def increment_iteration(self):
        """Incremente le compteur d'iterations."""
        self.iteration += 1
    
    def designate_next_agent(self, agent_name: str):
        """Designe l'agent qui doit intervenir ensuite."""
        self._next_agent = agent_name
    
    def consume_next_agent_designation(self) -> Optional[str]:
        """Retourne et efface la designation d'agent."""
        agent = self._next_agent
        self._next_agent = None
        return agent
    
    def get_state_snapshot(self, summarize: bool = True) -> Dict[str, Any]:
        """Retourne un snapshot de l'etat pour les plugins."""
        if summarize:
            return {
                "session_id": self.session_id,
                "theorem": self.theorem_statement,
                "goal": self.current_goal,
                "phase": self.phase.value,
                "strategy": self.strategy.value,
                "iteration": f"{self.iteration}/{self.max_iterations}",
                "proof_steps": len(self.current_proof),
                "discovered_lemmas": len(self.discovered_lemmas),
                "errors": self.error_count,
                "last_error": self.last_error
            }
        else:
            return self.to_dict()


    def add_verification(self, attempt_id: str, success: bool, output: str, errors: str,
                         remaining_goals: Optional[str] = None, exec_time_ms: float = 0.0,
                         mode: str = "subprocess") -> str:
        """Enregistre un résultat de vérification Lean."""
        verif_id = f"verif_{len(self.verification_results) + 1}"
        self.verification_results.append({
            "id": verif_id,
            "attempt_id": attempt_id,
            "success": success,
            "output": output,
            "errors": errors,
            "remaining_goals": remaining_goals,
            "exec_time_ms": exec_time_ms,
            "mode": mode,
            "timestamp": datetime.now().isoformat()
        })
        return verif_id


    def set_proof_complete(self, proof: str):
        """Marque la preuve comme terminée et change la phase."""
        self.final_proof = proof
        self.phase = ProofPhase.COMPLETE


    def set_strategy(self, strategy: 'ProofStrategy'):
        """Change la stratégie de preuve."""
        self.strategy = strategy

    def to_dict(self) -> Dict[str, Any]:
        """Serialise l'etat."""
        return {
            "session_id": self.session_id,
            "theorem_name": self.theorem_name,
            "theorem_statement": self.theorem_statement,
            "current_goal": self.current_goal,
            "current_proof": self.current_proof,
            "phase": self.phase.value,
            "strategy": self.strategy.value,
            "discovered_lemmas": self.discovered_lemmas,
            "generated_tactics": self.generated_tactics,
            "iteration": self.iteration,
            "max_iterations": self.max_iterations,
            "error_count": self.error_count,
            "last_error": self.last_error
        }

# --- Test de l'initialisation ---
print("\n" + "="*60)
print("ProofState initialise avec succes")
print(f"LeanRunner disponible: {LeanRunner is not None}")
print("="*60)


Configuration chargee depuis: /mnt/d/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean/.env
lean_runner importe avec succes depuis /mnt/d/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean

ProofState initialise avec succes
LeanRunner disponible: True


### 1.1. Vue d'ensemble des Plugins

L'architecture utilise 4 plugins specialises, chacun exposant des fonctions via `@kernel_function`:

| Plugin | Role | Fonctions cles |
|--------|------|----------------|
| **ProofStateManagerPlugin** | Gestion de l'etat | get_proof_state, add_lemma, designate_next_agent |
| **LeanSearchPlugin** | Recherche Mathlib | search_mathlib_lemmas, check_lemma_type |
| **LeanTacticPlugin** | Generation tactiques | generate_tactics, analyze_tactic_failure |
| **LeanVerificationPlugin** | Verification Lean | verify_proof, verify_tactic_step |

Ce pattern permet aux agents d'appeler ces fonctions automatiquement grace au `FunctionChoiceBehavior.Auto()` de Semantic Kernel.

## 🔌 Plugins Semantic Kernel : Exposer l'État aux Agents

### Problème

Les agents LLM ne peuvent pas accéder directement à `ProofState` (objet Python).

### Solution : Plugins

Un **plugin Semantic Kernel** expose des méthodes Python comme **fonctions appelables par le LLM**.

```python
@kernel_function(
    description="Enregistre une tentative de tactique",
    name="log_tactic_attempt"
)
def log_tactic_attempt(self, tactic: str, confidence: float) -> str:
    attempt_id = self._state.add_tactic_attempt(tactic, confidence=confidence)
    return f"Tactique {tactic} enregistrée avec ID {attempt_id}"
```

### Décorateur `@kernel_function`

- `description` : Ce que le LLM voit ("À quoi sert cette fonction ?")
- `name` : Nom de la fonction pour le LLM
- Paramètres : Doivent correspondre **EXACTEMENT** à ce que le plugin appelle

### Les 4 plugins

1. **log_tactic_attempt** : Enregistrer une tactique essayée
2. **add_verification_result** : Enregistrer le résultat Lean
3. **set_proof_strategy** : Changer la stratégie de recherche
4. **mark_proof_complete** : Déclarer la preuve terminée

### Pourquoi c'est critique ?

Sans plugins, le LLM ne peut que **parler** de preuves. Avec plugins, il peut **agir** :

- Essayer des tactiques
- Vérifier formellement
- Ajuster sa stratégie en temps réel

In [17]:
# =============================================================================
# Section 8.2-8.5 - Plugins Semantic Kernel
# =============================================================================
# Architecture en 4 plugins specialises:
# - ProofStateManagerPlugin: Gestion de l'etat partage
# - LeanSearchPlugin: Recherche de lemmes Mathlib
# - LeanTacticPlugin: Generation de tactiques
# - LeanVerificationPlugin: Verification avec lean_runner.py

# Import du decorateur kernel_function
try:
    from semantic_kernel.functions import kernel_function
    SK_AVAILABLE = True
    print("Semantic Kernel disponible - utilisation des vrais decorateurs")
except ImportError:
    SK_AVAILABLE = False
    print("Semantic Kernel non disponible - mode simulation")
    # Decorateur de simulation
    def kernel_function(description="", name=None):
        def decorator(func):
            func._sk_function = True
            func._sk_description = description
            func._sk_name = name or func.__name__
            return func
        return decorator

# =============================================================================
# 8.2 ProofStateManagerPlugin
# =============================================================================

class ProofStateManagerPlugin:
    """
    Plugin pour gerer l'etat partage de la preuve.
    Expose les methodes de ProofState via @kernel_function.
    """

    def __init__(self, state: ProofState):
        self._state = state

    @kernel_function(
        description="Obtient un apercu de l'etat actuel de la preuve (theoreme, lemmes, tactiques, etc.)",
        name="get_proof_state"
    )
    def get_proof_state(self, summarize: bool = True) -> str:
        """Retourne l'etat actuel sous forme JSON."""
        snapshot = self._state.get_state_snapshot(summarize=summarize)
        return json.dumps(snapshot, indent=2, ensure_ascii=False)

    @kernel_function(
        description="Ajoute un lemme decouvert a l'etat partage",
        name="add_discovered_lemma"
    )
    def add_discovered_lemma(
        self, name: str, statement: str, namespace: str = "", relevance: float = 0.5
    ) -> str:
        """Enregistre un lemme trouve par SearchAgent."""
        lemma_id = self._state.add_lemma(name, statement, namespace, relevance)
        return f"Lemme ajoute: {lemma_id} ({name})"

    @kernel_function(
        description="Enregistre une tentative de tactique avec son niveau de confiance",
        name="log_tactic_attempt"
    )
    def log_tactic_attempt(
        self, tactic: str, state_before: str, confidence: float = 0.5, explanation: str = ""
    ) -> str:
        """Enregistre une tactique tentee par TacticAgent."""
        attempt_id = self._state.add_tactic_attempt(tactic, state_before, confidence, explanation)
        return f"Tactique enregistree: {attempt_id}"

    @kernel_function(
        description="Enregistre le resultat d'une verification Lean",
        name="add_verification_result"
    )
    def add_verification_result(
        self, attempt_id: str, success: bool, output: str, errors: str,
        remaining_goals: str = "", exec_time_ms: float = 0.0
    ) -> str:
        """Enregistre un resultat de verification."""
        verif_id = self._state.add_verification(
            attempt_id, success, output, errors,
            remaining_goals if remaining_goals else None, exec_time_ms, "subprocess"
        )
        status = "OK" if success else "ECHEC"
        return f"Verification {verif_id}: {status}"

    @kernel_function(
        description="Designe l'agent qui doit parler au prochain tour. IMPORTANT: utiliser le nom exact.",
        name="designate_next_agent"
    )
    def designate_next_agent(self, agent_name: str) -> str:
        """Delegue au prochain agent."""
        valid_agents = ["SearchAgent", "TacticAgent", "VerifierAgent", "CriticAgent", "CoordinatorAgent"]
        if agent_name not in valid_agents:
            return f"ERREUR: Agent invalide '{agent_name}'. Valides: {valid_agents}"
        self._state.designate_next_agent(agent_name)
        return f"Prochain agent: {agent_name}"

    @kernel_function(
        description="Marque la preuve comme terminee avec le code final",
        name="set_proof_complete"
    )
    def set_proof_complete(self, proof_code: str) -> str:
        """Marque la preuve comme reussie."""
        self._state.set_proof_complete(proof_code)
        return f"PREUVE COMPLETE! Code: {proof_code[:100]}..."

    @kernel_function(
        description="Change la strategie de preuve (exploration, refinement, validation, recovery)",
        name="set_proof_strategy"
    )
    def set_proof_strategy(self, strategy: str) -> str:
        """Change la strategie de preuve."""
        try:
            self._state.set_strategy(ProofStrategy(strategy))
            return f"Strategie changee: {strategy}"
        except ValueError:
            return f"ERREUR: Strategie invalide '{strategy}'. Valides: exploration, refinement, validation, recovery"


Semantic Kernel disponible - utilisation des vrais decorateurs


### 1.2. LeanSearchPlugin : Recherche de Lemmes Mathlib

**Plugin exposant les méthodes de recherche** pour SearchAgent.

#### Méthodes Exposées

```python
@kernel_function
def search_lemmas(goal: str, keywords: List[str]) -> List[Lemma]:
    # Recherche dans Mathlib par keywords
    # Retourne lemmes triés par pertinence
```

**Pattern** : SearchAgent appelle ce plugin pour trouver lemmes Mathlib pertinents.


In [18]:


# =============================================================================
# 8.3 LeanSearchPlugin
# =============================================================================

class LeanSearchPlugin:
    """
    Plugin pour la recherche de lemmes dans Mathlib.
    Utilise des patterns connus + verification #check via lean_runner.
    """

    def __init__(self, runner: LeanRunner):
        self._runner = runner
        # Base de lemmes connus (extensible)
        self._known_lemmas = {
            # Arithmetique de base
            "Nat.add_zero": ("n + 0 = n", "Nat"),
            "Nat.zero_add": ("0 + n = n", "Nat"),
            "Nat.add_comm": ("n + m = m + n", "Nat"),
            "Nat.add_assoc": ("(n + m) + k = n + (m + k)", "Nat"),
            "Nat.mul_one": ("n * 1 = n", "Nat"),
            "Nat.one_mul": ("1 * n = n", "Nat"),
            "Nat.mul_comm": ("n * m = m * n", "Nat"),
            "Nat.mul_assoc": ("(n * m) * k = n * (m * k)", "Nat"),
            "Nat.left_distrib": ("n * (m + k) = n * m + n * k", "Nat"),
            "Nat.right_distrib": ("(n + m) * k = n * k + m * k", "Nat"),
            # Logique
            "And.intro": ("a -> b -> a /\\ b", "Logic"),
            "And.left": ("a /\\ b -> a", "Logic"),
            "And.right": ("a /\\ b -> b", "Logic"),
            "Or.inl": ("a -> a \\/ b", "Logic"),
            "Or.inr": ("b -> a \\/ b", "Logic"),
            "Eq.refl": ("a = a", "Logic"),
            "Eq.symm": ("a = b -> b = a", "Logic"),
            "Eq.trans": ("a = b -> b = c -> a = c", "Logic"),
        }

    @kernel_function(
        description="Recherche des lemmes Mathlib pertinents pour un but donne",
        name="search_mathlib_lemmas"
    )
    def search_mathlib_lemmas(self, goal: str, max_results: int = 10) -> str:
        """
        Recherche des lemmes par mots-cles.

        Args:
            goal: Description du but ou mots-cles (ex: "addition commutative")
            max_results: Nombre maximum de resultats

        Returns:
            JSON avec les lemmes trouves
        """
        goal_lower = goal.lower()
        results = []

        # Recherche par mots-cles
        keywords = goal_lower.replace("+", "add").replace("*", "mul").replace("=", "eq").split()

        for name, (statement, namespace) in self._known_lemmas.items():
            score = 0.0
            name_lower = name.lower()

            # Scoring par mots-cles
            for kw in keywords:
                if kw in name_lower:
                    score += 0.3
                if kw in statement.lower():
                    score += 0.2

            # Patterns specifiques
            if "comm" in goal_lower and "comm" in name_lower:
                score += 0.4
            if "assoc" in goal_lower and "assoc" in name_lower:
                score += 0.4
            if "zero" in goal_lower and "zero" in name_lower:
                score += 0.3
            if "distrib" in goal_lower and "distrib" in name_lower:
                score += 0.4

            if score > 0:
                results.append({
                    "name": name,
                    "statement": statement,
                    "namespace": namespace,
                    "relevance": min(score, 1.0)
                })

        # Trier par pertinence
        results.sort(key=lambda x: x["relevance"], reverse=True)
        return json.dumps(results[:max_results], indent=2, ensure_ascii=False)

    @kernel_function(
        description="Verifie qu'un lemme existe et retourne son type via #check",
        name="check_lemma_type"
    )
    def check_lemma_type(self, lemma_name: str) -> str:
        """
        Verifie l'existence d'un lemme via #check.

        Args:
            lemma_name: Nom du lemme (ex: "Nat.add_comm")

        Returns:
            JSON {exists, type, error}
        """
        code = f"#check {lemma_name}"
        result = self._runner.run(code)

        if result.success and not result.errors:
            # Extraire le type de la sortie
            return json.dumps({
                "exists": True,
                "type": result.output.strip(),
                "error": None
            })
        else:
            return json.dumps({
                "exists": False,
                "type": None,
                "error": result.errors or "Lemme non trouve"
            })


# =============================================================================
# 8.4 LeanTacticPlugin


### 1.3. Plugins de Tactiques et Verification

Les deux plugins restants gerent la **generation de tactiques** et la **verification formelle** :

#### LeanTacticPlugin

- **Responsabilite** : Generer des tactiques Lean adaptees au contexte
- **Methodes exposees** :
  - `generate_tactic()` : Genere une tactique basee sur goal + lemmes + historique
  - `estimate_confidence()` : Estime la probabilite de succes (0.0-1.0)
- **LLM-aware** : Utilise un prompt structure pour le LLM avec exemples de tactiques Lean
- **Strategies** : `exact`, `rw`, `apply`, `simp`, `induction`, `cases`, etc.

#### LeanVerificationPlugin

- **Responsabilite** : Verifier les preuves via compilation Lean
- **Methodes exposees** :
  - `verify_proof()` : Compile le theoreme avec tactiques et retourne succes/echec
  - `parse_lean_errors()` : Parse les messages d'erreur Lean pour feedback agents
- **Detection de completion** : Reconnait "no goals" = preuve complete
- **Gestion d'erreurs** : Extrait type d'erreur (type mismatch, tactic failed, etc.) pour CriticAgent

**Flow typique** :
```
SearchAgent trouve lemmes
   |
   v
TacticAgent genere tactique (via LeanTacticPlugin)
   |
   v
VerifierAgent compile (via LeanVerificationPlugin)
   |
   +-- Success → COMPLETE
   +-- Failure → CriticAgent analyse → retry
```


In [19]:
# =============================================================================

class LeanTacticPlugin:
    """
    Plugin pour la generation de tactiques.
    Fournit des heuristiques et analyse les echecs.
    """

    def __init__(self):
        # Tactiques par difficulte
        self._tactics = {
            "simple": ["rfl", "trivial", "exact ?_", "assumption"],
            "medium": ["simp", "omega", "decide", "constructor", "intro", "apply"],
            "complex": ["ring", "linarith", "aesop", "induction", "cases", "rcases"]
        }

        # Heuristiques par pattern de but
        self._heuristics = {
            "equality": ["rfl", "exact", "simp", "ring", "omega"],
            "forall": ["intro", "intros", "apply"],
            "exists": ["use", "exists", "exact"],
            "and": ["constructor", "exact And.intro"],
            "or": ["left", "right"],
            "implication": ["intro", "apply", "exact"],
            "nat_arithmetic": ["omega", "simp", "decide"],
            "ring_expression": ["ring", "ring_nf"]
        }

    @kernel_function(
        description="Genere des tactiques appropriees pour un but donne",
        name="generate_tactics"
    )
    def generate_tactics(self, goal: str, context: str = "", difficulty: str = "simple") -> str:
        """
        Genere des tactiques pour le but courant.

        Args:
            goal: Le but Lean a prouver
            context: Contexte additionnel (lemmes disponibles, etc.)
            difficulty: simple, medium, ou complex

        Returns:
            JSON [{tactic, confidence, explanation}]
        """
        suggestions = []
        goal_lower = goal.lower()

        # Detecter le type de but
        detected_patterns = []
        if "=" in goal:
            detected_patterns.append("equality")
        if "forall" in goal_lower or "∀" in goal:
            detected_patterns.append("forall")
        if "exists" in goal_lower or "∃" in goal:
            detected_patterns.append("exists")
        if "/\\" in goal or "∧" in goal or "And" in goal:
            detected_patterns.append("and")
        if "\\/" in goal or "∨" in goal or "Or" in goal:
            detected_patterns.append("or")
        if "->" in goal or "→" in goal:
            detected_patterns.append("implication")
        if any(x in goal_lower for x in ["nat", "n +", "m +", "+ 0", "0 +"]):
            detected_patterns.append("nat_arithmetic")
        if any(x in goal for x in ["*", "+"]) and "=" in goal:
            detected_patterns.append("ring_expression")

        # Collecter les tactiques suggeres
        seen = set()
        for pattern in detected_patterns:
            for tactic in self._heuristics.get(pattern, []):
                if tactic not in seen:
                    seen.add(tactic)
                    confidence = 0.7 if difficulty == "simple" else 0.5
                    suggestions.append({
                        "tactic": tactic,
                        "confidence": confidence,
                        "explanation": f"Pattern detecte: {pattern}"
                    })

        # Ajouter des tactiques de base
        base_tactics = self._tactics.get(difficulty, self._tactics["simple"])
        for tactic in base_tactics[:3]:
            if tactic not in seen:
                suggestions.append({
                    "tactic": tactic,
                    "confidence": 0.3,
                    "explanation": f"Tactique {difficulty} generique"
                })

        # Trier par confiance
        suggestions.sort(key=lambda x: x["confidence"], reverse=True)
        return json.dumps(suggestions[:8], indent=2, ensure_ascii=False)

    @kernel_function(
        description="Analyse un echec de tactique et suggere des alternatives",
        name="analyze_tactic_failure"
    )
    def analyze_tactic_failure(self, failed_tactic: str, error_msg: str) -> str:
        """
        Analyse pourquoi une tactique a echoue.

        Args:
            failed_tactic: La tactique qui a echoue
            error_msg: Message d'erreur Lean

        Returns:
            JSON {diagnosis, alternatives, error_type}
        """
        error_lower = error_msg.lower()
        diagnosis = ""
        alternatives = []
        error_type = "unknown"

        # Classifier l'erreur
        if "unknown identifier" in error_lower or "unknown constant" in error_lower:
            error_type = "unknown_identifier"
            diagnosis = "Lemme ou identifiant non reconnu. Verifier l'import ou le nom."
            alternatives = ["Chercher le bon nom avec #check", "Verifier les imports"]

        elif "type mismatch" in error_lower:
            error_type = "type_mismatch"
            diagnosis = "Les types ne correspondent pas. Verifier les arguments."
            alternatives = ["exact", "apply", "simp"]

        elif "unsolved goals" in error_lower or "goals remain" in error_lower:
            error_type = "unsolved_goals"
            diagnosis = "Des sous-buts restent. La tactique n'a pas complete la preuve."
            alternatives = ["Ajouter d'autres tactiques", "Essayer simp", "Decomposer avec have"]

        elif "tactic failed" in error_lower:
            error_type = "tactic_failed"
            diagnosis = f"La tactique '{failed_tactic}' n'a pas pu s'appliquer."
            # Suggerer des alternatives
            if failed_tactic in ["ring", "linarith"]:
                alternatives = ["omega", "simp", "decide"]
            elif failed_tactic == "simp":
                alternatives = ["simp only", "rfl", "exact"]
            else:
                alternatives = ["simp", "omega", "exact ?_"]

        elif "declaration uses 'sorry'" in error_lower:
            error_type = "sorry"
            diagnosis = "La preuve contient 'sorry' - incomplete."
            alternatives = ["Completer la preuve", "Remplacer sorry par une vraie tactique"]

        else:
            error_type = "other"
            diagnosis = f"Erreur non classifiee: {error_msg[:100]}"
            alternatives = ["Verifier la syntaxe", "Essayer une approche differente"]

        return json.dumps({
            "diagnosis": diagnosis,
            "alternatives": alternatives,
            "error_type": error_type,
            "original_error": error_msg[:200]
        }, indent=2, ensure_ascii=False)


### 1.4. LeanVerificationPlugin : Compilation et Vérification

**Plugin exposant les méthodes de vérification** pour VerifierAgent.

#### Méthodes Exposées

```python
@kernel_function
def verify_proof(theorem: str, tactics: str) -> VerificationResult:
    # Compile le théorème avec tactiques
    # Parse output Lean (success/errors)
    # Détecte "no goals" = preuve complète
```

**Pattern** : VerifierAgent appelle ce plugin pour compiler preuves avec LeanRunner.


In [20]:


# =============================================================================
# 8.5 LeanVerificationPlugin
# =============================================================================

class LeanVerificationPlugin:
    """
    Plugin pour la verification des preuves avec lean_runner.
    """

    def __init__(self, runner: LeanRunner):
        self._runner = runner

    @kernel_function(
        description="Verifie une preuve complete (theoreme + tactiques)",
        name="verify_proof"
    )
    def verify_proof(self, theorem_statement: str, proof_tactics: str) -> str:
        """
        Verifie un theoreme avec sa preuve.

        Args:
            theorem_statement: L'enonce du theoreme (ex: "theorem add_zero (n : Nat) : n + 0 = n")
            proof_tactics: La preuve (ex: "exact Nat.add_zero n")

        Returns:
            JSON {success, output, errors, exec_time_ms, backend}
        """
        import time

        # Construire le code complet
        if "by" not in proof_tactics and ":=" not in proof_tactics:
            code = f"{theorem_statement} := by {proof_tactics}"
        elif ":=" in proof_tactics:
            code = f"{theorem_statement} {proof_tactics}"
        else:
            code = f"{theorem_statement} := {proof_tactics}"

        start = time.time()
        result = self._runner.run(code)
        exec_time = (time.time() - start) * 1000

        return json.dumps({
            "success": result.success,
            "output": result.output,
            "errors": result.errors,
            "exit_code": result.exit_code,
            "exec_time_ms": round(exec_time, 2),
            "backend": result.backend,
            "code": code
        }, indent=2, ensure_ascii=False)

    @kernel_function(
        description="Verifie une etape de tactique incrementale",
        name="verify_tactic_step"
    )
    def verify_tactic_step(
        self, partial_proof: str, next_tactic: str, theorem_statement: str
    ) -> str:
        """
        Verifie une tactique incrementale.

        Args:
            partial_proof: Les tactiques deja appliquees (separees par ;)
            next_tactic: La prochaine tactique a essayer
            theorem_statement: L'enonce du theoreme

        Returns:
            JSON {tactic_valid, remaining_goals, error, exec_time_ms}
        """
        import time

        # Combiner les tactiques
        if partial_proof:
            all_tactics = f"{partial_proof}; {next_tactic}"
        else:
            all_tactics = next_tactic

        code = f"{theorem_statement} := by {all_tactics}"

        start = time.time()
        result = self._runner.run(code)
        exec_time = (time.time() - start) * 1000

        # Analyser les goals restants
        remaining_goals = None
        if "unsolved goals" in result.errors.lower():
            # Extraire les goals du message d'erreur
            remaining_goals = result.errors

        return json.dumps({
            "tactic_valid": result.success or "unsolved goals" not in result.errors.lower(),
            "remaining_goals": remaining_goals,
            "error": result.errors if not result.success else None,
            "exec_time_ms": round(exec_time, 2),
            "applied_tactics": all_tactics
        }, indent=2, ensure_ascii=False)


# =============================================================================
# Test des Plugins
# =============================================================================

print("\n=== Test des Plugins ===")

# Creer l'etat et le runner
test_state = ProofState(theorem_statement="theorem test_add (n : Nat) : n + 0 = n")
runner = LeanRunner(backend="subprocess", timeout=30)

# Instancier les plugins
state_plugin = ProofStateManagerPlugin(test_state)
search_plugin = LeanSearchPlugin(runner)
tactic_plugin = LeanTacticPlugin()
verif_plugin = LeanVerificationPlugin(runner)

# Test 1: Recherche de lemmes
print("\n1. Recherche de lemmes pour 'addition zero':")
lemmas = search_plugin.search_mathlib_lemmas("addition zero", max_results=3)
print(lemmas)

# Test 2: Generation de tactiques
print("\n2. Tactiques pour 'n + 0 = n':")
tactics = tactic_plugin.generate_tactics("n + 0 = n", difficulty="simple")
print(tactics)

# Test 3: Verification avec lean_runner
print("\n3. Verification d'une preuve:")
result = verif_plugin.verify_proof("theorem test_rfl : 2 + 2 = 4", "rfl")
print(result)

# Test 4: Plugin StateManager
print("\n4. Ajout via StateManagerPlugin:")
print(state_plugin.add_discovered_lemma("Nat.add_zero", "n + 0 = n", "Nat", 0.9))
print(state_plugin.get_proof_state(summarize=True))



=== Test des Plugins ===

1. Recherche de lemmes pour 'addition zero':
[
  {
    "name": "Nat.add_zero",
    "statement": "n + 0 = n",
    "namespace": "Nat",
    "relevance": 0.6
  },
  {
    "name": "Nat.zero_add",
    "statement": "0 + n = n",
    "namespace": "Nat",
    "relevance": 0.6
  }
]

2. Tactiques pour 'n + 0 = n':
[
  {
    "tactic": "rfl",
    "confidence": 0.7,
    "explanation": "Pattern detecte: equality"
  },
  {
    "tactic": "exact",
    "confidence": 0.7,
    "explanation": "Pattern detecte: equality"
  },
  {
    "tactic": "simp",
    "confidence": 0.7,
    "explanation": "Pattern detecte: equality"
  },
  {
    "tactic": "ring",
    "confidence": 0.7,
    "explanation": "Pattern detecte: equality"
  },
  {
    "tactic": "omega",
    "confidence": 0.7,
    "explanation": "Pattern detecte: equality"
  },
  {
    "tactic": "decide",
    "confidence": 0.7,
    "explanation": "Pattern detecte: nat_arithmetic"
  },
  {
    "tactic": "ring_nf",
    "confidence": 0.7,


## 2. Architecture : 5 Agents Specialises

Le systeme multi-agents comprend 5 roles distincts:

| Agent | Role | Plugins | Delegation |
|-------|------|---------|------------|
| **SearchAgent** | Recherche lemmes Mathlib | LeanSearch, StateManager | TacticAgent si lemmes trouves |
| **TacticAgent** | Generation tactiques | LeanTactic, StateManager | VerifierAgent pour validation |
| **VerifierAgent** | Verification Lean | LeanVerification, StateManager | CriticAgent si echec |
| **CriticAgent** | Analyse echecs | LeanTactic, StateManager | Redirection selon erreur |
| **CoordinatorAgent** | Supervision globale | StateManager | Gestion des blocages |

**Pattern cle**: Chaque agent designe explicitement le suivant via `designate_next_agent()`.

## 🤖 Création des Agents Semantic Kernel

### Anatomie d'un agent

Chaque agent a :

1. **Un nom** : "SearchAgent", "TacticAgent", etc.
2. **Des instructions** : Prompt système qui définit son rôle
3. **Des plugins** : Fonctions qu'il peut appeler (via StatePlugin)
4. **Un modèle LLM** : GPT-5.2, Claude, etc.

### Exemple : SearchAgent

```python
search_agent = kernel.add_agent(
    name="SearchAgent",
    instructions="""Tu es un expert en recherche de lemmes Mathlib.
    Ton rôle : Trouver les lemmes pertinents pour le but actuel.
    Délègue à TacticAgent une fois les lemmes trouvés.""",
    plugins=[state_plugin]
)
```

### Instructions : Le "métier" de l'agent

Les instructions définissent :

- **Responsabilité** : "Recherche de lemmes" vs "Génération de tactiques"
- **Critères de succès** : "Trouver au moins 2 lemmes pertinents"
- **Délégation** : "Quand déléguer à un autre agent ?"

**Principe clé** : Instructions précises → Comportement prévisible

### Pattern : Stratégies basées sur l'état

Au lieu de coder en dur "SearchAgent → TacticAgent", on utilise :

```python
def select_next_agent(state: ProofState) -> str:
    if state.phase == ProofPhase.SEARCH:
        return "SearchAgent"
    elif state.phase == ProofPhase.TACTIC_GEN:
        return "TacticAgent"
    # ...
```

**Avantage** : Orchestration dynamique basée sur l'état réel de la preuve.

In [21]:
# =============================================================================
# Section 8.6 - Definition des 5 Agents Specialises avec Semantic Kernel
# =============================================================================
# Utilise ChatCompletionAgent de Semantic Kernel avec FunctionChoiceBehavior.Auto()

import os
import asyncio
from typing import Dict, Any, Optional

# --- Instructions des Agents ---

SEARCH_AGENT_INSTRUCTIONS = """
Tu es l'agent de RECHERCHE de lemmes pour le theorem proving en Lean 4.

TON ROLE UNIQUE:
- Chercher des lemmes Mathlib pertinents pour le theoreme courant
- Identifier les lemmes qui peuvent aider a la preuve
- Enregistrer les lemmes trouves dans l'etat partage

WORKFLOW:
1. Lis l'etat avec get_proof_state() pour comprendre le theoreme
2. Utilise search_mathlib_lemmas() avec des mots-cles pertinents
3. Verifie les lemmes prometteurs avec check_lemma_type()
4. Enregistre les lemmes utiles avec add_discovered_lemma()
5. Delegue a TacticAgent quand tu as trouve des lemmes

IMPORTANT:
- Cherche des lemmes LIES au but (egalites, arithmetique, logique)
- Delegation: Apres avoir trouve au moins 2-3 lemmes, delegue a TacticAgent
- Si aucun lemme pertinent, delegue quand meme a TacticAgent
"""

TACTIC_AGENT_INSTRUCTIONS = """
Tu es l'agent de GENERATION DE TACTIQUES pour le theorem proving en Lean 4.

TON ROLE UNIQUE:
- Generer des sequences de tactiques Lean pour prouver le but
- Utiliser les lemmes trouves par SearchAgent
- Proposer des tactiques avec niveau de confiance

WORKFLOW:
1. Lis l'etat avec get_proof_state() pour voir le theoreme et les lemmes
2. Utilise generate_tactics() pour obtenir des suggestions
3. Enregistre ta meilleure tentative avec log_tactic_attempt()
4. Delegue a VerifierAgent pour verification

STRATEGIES DE TACTIQUES (par difficulte):
- SIMPLE: rfl, trivial, exact, assumption
- MEDIUM: simp, omega, constructor, intro, apply
- COMPLEX: ring, linarith, induction, cases

IMPORTANT:
- Commence par les tactiques simples (rfl, exact)
- Utilise les lemmes trouves par SearchAgent (exact Nat.add_zero n)
- Delegation: TOUJOURS deleguer a VerifierAgent apres avoir propose
"""

VERIFIER_AGENT_INSTRUCTIONS = """
Tu es l'agent de VERIFICATION pour le theorem proving en Lean 4.

TON ROLE UNIQUE:
- Verifier les tactiques proposees avec le compilateur Lean
- Enregistrer les resultats de verification
- Determiner si la preuve est complete ou s'il faut continuer

WORKFLOW:
1. Lis l'etat avec get_proof_state() pour voir la derniere tactique
2. Utilise verify_proof() pour tester la preuve
3. Enregistre le resultat avec add_verification_result()
4. Si succes: set_proof_complete() et termine
5. Si echec: delegue a CriticAgent pour analyse

IMPORTANT:
- Teste TOUJOURS la derniere tactique proposee
- Si la preuve compile sans erreur, utilise set_proof_complete()
- Si echec, enregistre l'erreur et delegue a CriticAgent
"""

CRITIC_AGENT_INSTRUCTIONS = """
Tu es l'agent CRITIQUE pour le theorem proving en Lean 4.

TON ROLE UNIQUE:
- Analyser les echecs de verification
- Diagnostiquer les erreurs Lean
- Orienter vers la bonne strategie de correction

WORKFLOW:
1. Lis l'etat avec get_proof_state() pour voir les echecs recents
2. Utilise analyze_tactic_failure() pour comprendre l'erreur
3. Decide quelle direction prendre:
   - "unknown identifier" -> delegue a SearchAgent
   - "type mismatch" ou "tactic failed" -> delegue a TacticAgent
   - Echecs repetes (>3) -> delegue a CoordinatorAgent

IMPORTANT:
- Analyse les 3 derniers echecs pour detecter des patterns
- Si >3 echecs similaires, delegue a CoordinatorAgent
"""

COORDINATOR_AGENT_INSTRUCTIONS = """
Tu es l'agent COORDINATEUR (superviseur) pour le theorem proving en Lean 4.

TON ROLE UNIQUE:
- Superviser l'ensemble de la session de preuve
- Debloquer les situations cycliques
- Ajuster la strategie globale

QUAND TU INTERVIENS:
- Appele par CriticAgent apres echecs repetes
- Appele si max_iterations approche
- Appele pour decisions strategiques majeures

IMPORTANT:
- Tu es le dernier recours, prends des decisions audacieuses
- Si >40 iterations, suggere de simplifier le theoreme
"""

# =============================================================================
# Detection de Semantic Kernel
# =============================================================================

SK_AVAILABLE = False
try:
    from semantic_kernel import Kernel
    from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
    from semantic_kernel.agents.strategies import (
        KernelFunctionSelectionStrategy,
        KernelFunctionTerminationStrategy,
    )
    from semantic_kernel.agents.strategies.selection.sequential_selection_strategy import SequentialSelectionStrategy
    from semantic_kernel.agents.strategies.termination.default_termination_strategy import DefaultTerminationStrategy
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
    from semantic_kernel.connectors.ai import FunctionChoiceBehavior
    from semantic_kernel.functions import KernelFunctionFromPrompt, KernelArguments
    from semantic_kernel.contents import ChatHistoryTruncationReducer
    from semantic_kernel.agents.strategies.selection.selection_strategy import SelectionStrategy
    from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
    from semantic_kernel.contents.chat_message_content import ChatMessageContent
    from pydantic import PrivateAttr
    SK_AVAILABLE = True
    print("Semantic Kernel disponible - utilisation de ChatCompletionAgent")
except ImportError as e:
    print(f"Semantic Kernel non disponible: {e}")
    print("Installation: pip install semantic-kernel")


Semantic Kernel disponible - utilisation de ChatCompletionAgent


### 2.1. SimpleAgent : Agent Fallback : Agent Fallback (Simulation)

**Classe de secours** pour simuler agents quand Semantic Kernel non disponible.

#### Architecture

```python
class SimpleAgent:
    def __init__(self, name: str, instructions: str):
        self.name = name
        self.instructions = instructions
    
    def invoke(self, message: str) -> str:
        # Simulation simple basée sur règles
        # Utilisé en mode fallback si OpenAI API indisponible
```

**Usage** : Mode simulation pour tests sans LLM.


In [22]:

# =============================================================================
# Mode Simulation (fallback si SK non disponible)
# =============================================================================

class SimpleAgent:
    """
    Agent simplifie pour simulation ou fallback.
    Utilise directement l'API OpenAI si SK n'est pas disponible.
    """

    def __init__(
        self,
        name: str,
        instructions: str,
        plugins: Dict[str, Any],
        use_simulation: bool = True
    ):
        self.name = name
        self.instructions = instructions
        self.plugins = plugins
        self.use_simulation = use_simulation
        self._openai_client = None

        # Initialiser le client OpenAI si mode reel
        if not use_simulation:
            try:
                from openai import OpenAI
                api_key = os.getenv("OPENAI_API_KEY")
                if api_key and len(api_key) > 10 and not api_key.startswith("sk-..."):
                    self._openai_client = OpenAI(api_key=api_key)
            except ImportError:
                pass

    def _build_openai_tools(self) -> list:
        """Construit les outils au format OpenAI function calling."""
        import inspect
        tools = []
        for plugin_name, plugin in self.plugins.items():
            for attr_name in dir(plugin):
                attr = getattr(plugin, attr_name)
                if not callable(attr):
                    continue
                # Supporter les deux décorateurs
                is_sk_func = hasattr(attr, '_sk_function') or hasattr(attr, '__kernel_function__')
                if not is_sk_func:
                    continue

                sig = inspect.signature(attr)
                properties = {}
                required = []
                for param_name, param in sig.parameters.items():
                    if param_name == 'self':
                        continue
                    param_type = "string"
                    if param.annotation != inspect.Parameter.empty:
                        if param.annotation == bool:
                            param_type = "boolean"
                        elif param.annotation in (int, float):
                            param_type = "number"
                    properties[param_name] = {
                        "type": param_type,
                        "description": f"Parameter {param_name}"
                    }
                    if param.default == inspect.Parameter.empty:
                        required.append(param_name)

                # Obtenir nom et description
                if hasattr(attr, '__kernel_function_name__'):
                    func_name = attr.__kernel_function_name__
                    func_desc = getattr(attr, "__kernel_function_description__", "")
                elif hasattr(attr, '_sk_name'):
                    func_name = attr._sk_name
                    func_desc = getattr(attr, "_sk_description", "")
                else:
                    func_name = attr_name
                    func_desc = ""

                tools.append({
                    "type": "function",
                    "function": {
                        "name": f"{plugin_name}__{func_name}",
                        "description": func_desc,
                        "parameters": {
                            "type": "object",
                            "properties": properties,
                            "required": required
                        }
                    }
                })
        return tools

    def _execute_tool_call(self, tool_name: str, arguments: dict) -> str:
        """Execute un appel de fonction sur un plugin."""
        parts = tool_name.split("__", 1)
        if len(parts) != 2:
            return f"Erreur: format invalide: {tool_name}"

        plugin_name, func_name = parts
        plugin = self.plugins.get(plugin_name)
        if not plugin:
            return f"Erreur: plugin {plugin_name} non trouve"

        for attr_name in dir(plugin):
            attr = getattr(plugin, attr_name)
            if not callable(attr):
                continue
            is_sk = hasattr(attr, '_sk_function') or hasattr(attr, '__kernel_function__')
            if not is_sk:
                continue

            if hasattr(attr, '__kernel_function_name__'):
                name = attr.__kernel_function_name__
            elif hasattr(attr, '_sk_name'):
                name = attr._sk_name
            else:
                name = attr_name

            if name == func_name:
                try:
                    result = attr(**arguments)
                    return str(result)
                except Exception as e:
                    return f"Erreur {func_name}: {e}"

        return f"Erreur: {func_name} non trouve dans {plugin_name}"

    def invoke(self, message: str, state: ProofState) -> str:
        """Execute l'agent sur un message."""
        state.increment_iteration()

        if self.use_simulation or not self._openai_client:
            return self._simulate_response(message, state)
        else:
            return self._call_llm(message, state)

    def _simulate_response(self, message: str, state: ProofState) -> str:
        """
        Simulation realiste basee sur l'analyse du theoreme.
        La complexite vient du nombre de lemmes necessaires.
        """
        theorem = state.theorem_statement.lower()
        goal = state.current_goal or ""

        if self.name == "SearchAgent":
            return self._do_search(state, theorem, goal)
        elif self.name == "TacticAgent":
            return self._do_tactic(state, theorem, goal)
        elif self.name == "VerifierAgent":
            return self._do_verify(state, theorem, goal)
        elif self.name == "CriticAgent":
            return self._do_critic(state, theorem)
        elif self.name == "CoordinatorAgent":
            return self._do_coordinate(state, theorem)

        return f"[{self.name}] Action simulee."

    def _do_search(self, state: ProofState, theorem: str, goal: str) -> str:
        """Recherche de lemmes basee sur le theoreme."""
        search = self.plugins.get("search")
        state_mgr = self.plugins.get("state")

        if not search or not state_mgr:
            return "[SearchAgent] Plugins manquants."

        lemmas_found = []

        if "n = n" in theorem or goal.strip() == "n = n":
            state_mgr.add_discovered_lemma("Eq.refl", "a = a", "Logic", 1.0)
            lemmas_found.append("Eq.refl")

        if "+ 0" in theorem or "0 +" in theorem:
            state_mgr.add_discovered_lemma("Nat.add_zero", "n + 0 = n", "Nat", 0.9)
            lemmas_found.append("Nat.add_zero")

        if "+" in theorem and ("m + n" in theorem or "n + m" in theorem or "b + a" in theorem or "d + c" in theorem):
            state_mgr.add_discovered_lemma("Nat.add_comm", "n + m = m + n", "Nat", 0.85)
            lemmas_found.append("Nat.add_comm")

        if theorem.count("+") >= 2:
            state_mgr.add_discovered_lemma("Nat.add_assoc", "(n + m) + k = n + (m + k)", "Nat", 0.8)
            lemmas_found.append("Nat.add_assoc")

        if "*" in theorem and "+" in theorem:
            state_mgr.add_discovered_lemma("Nat.right_distrib", "(n + m) * k = n * k + m * k", "Nat", 0.9)
            lemmas_found.append("Nat.right_distrib")

        state_mgr.designate_next_agent("TacticAgent")
        return f"[SearchAgent] Lemmes: {', '.join(lemmas_found[:3])}. -> TacticAgent"

    def _do_tactic(self, state: ProofState, theorem: str, goal: str) -> str:
        """Generation de tactiques."""
        state_mgr = self.plugins.get("state")
        if not state_mgr:
            return "[TacticAgent] Plugin manquant."

        tactics_tried = len(state.tactics_history)

        # DEMO_1
        if "n = n" in theorem:
            state_mgr.log_tactic_attempt("rfl", goal, 1.0, "Reflexivite")
            state_mgr.designate_next_agent("VerifierAgent")
            return "[TacticAgent] Tactique: rfl. -> VerifierAgent"

        # DEMO_2
        if "n + m + 0 = m + n" in theorem:
            if tactics_tried == 0:
                state_mgr.log_tactic_attempt("simp [Nat.add_zero]", goal, 0.6, "Etape 1")
                state_mgr.designate_next_agent("VerifierAgent")
                return "[TacticAgent] Etape 1: simp [Nat.add_zero]. -> VerifierAgent"
            else:
                state_mgr.log_tactic_attempt("simp [Nat.add_zero, Nat.add_comm]", goal, 0.95, "Complete")
                state_mgr.designate_next_agent("VerifierAgent")
                return "[TacticAgent] Tactique: simp [Nat.add_zero, Nat.add_comm]. -> VerifierAgent"

        # DEMO_3
        if "quad_comm" in theorem:
            if tactics_tried < 3:
                tactic = ["rw [Nat.add_comm c d]", "rw [Nat.add_comm a b]", "rw [Nat.add_comm]"][min(tactics_tried, 2)]
                state_mgr.log_tactic_attempt(tactic, goal, 0.5 + tactics_tried * 0.1, f"Etape {tactics_tried+1}")
                state_mgr.designate_next_agent("VerifierAgent")
                return f"[TacticAgent] Etape {tactics_tried+1}/4: {tactic}. -> VerifierAgent"
            else:
                state_mgr.log_tactic_attempt("simp only [Nat.add_comm, Nat.add_assoc, Nat.add_left_comm]", goal, 0.95, "AC norm")
                state_mgr.designate_next_agent("VerifierAgent")
                return "[TacticAgent] Tactique finale: AC normalization. -> VerifierAgent"

        # DEMO_4
        if "distrib_both" in theorem:
            tactics = ["rw [Nat.right_distrib]", "rw [Nat.add_assoc]", "rw [Nat.add_comm (a*c)]", "rw [<-Nat.add_assoc]", "ring"]
            if tactics_tried < len(tactics):
                t = tactics[tactics_tried]
                state_mgr.log_tactic_attempt(t, goal, 0.5 + tactics_tried * 0.1, f"Etape {tactics_tried+1}")
                state_mgr.designate_next_agent("VerifierAgent")
                return f"[TacticAgent] Etape {tactics_tried+1}/5: {t}. -> VerifierAgent"
            else:
                state_mgr.log_tactic_attempt("ring", goal, 0.95, "Solveur")
                state_mgr.designate_next_agent("VerifierAgent")
                return "[TacticAgent] Tactique finale: ring. -> VerifierAgent"

        state_mgr.log_tactic_attempt("simp", goal, 0.3, "Fallback")
        state_mgr.designate_next_agent("VerifierAgent")
        return "[TacticAgent] simp. -> VerifierAgent"

    def _do_verify(self, state: ProofState, theorem: str, goal: str) -> str:
        """Verification."""
        state_mgr = self.plugins.get("state")
        if not state_mgr or not state.tactics_history:
            return "[VerifierAgent] Rien a verifier."

        last = state.tactics_history[-1]
        n = len(state.tactics_history)
        attempt_id = f"attempt_{n}"

        # DEMO_1
        if "rfl" in last.tactic and "n = n" in theorem:
            state_mgr.add_verification_result(attempt_id, True, "OK", "", "", 50.0)
            state_mgr.set_proof_complete(last.tactic)
            return f"[VerifierAgent] SUCCES! {last.tactic}"

        # DEMO_2
        if "n + m + 0 = m + n" in theorem:
            if n == 1:
                state_mgr.add_verification_result(attempt_id, False, "n+m = m+n", "not closed", "", 80.0)
                state_mgr.designate_next_agent("CriticAgent")
                return "[VerifierAgent] Progres. But restant. -> CriticAgent"
            else:
                state_mgr.add_verification_result(attempt_id, True, "OK", "", "", 100.0)
                state_mgr.set_proof_complete(last.tactic)
                return f"[VerifierAgent] SUCCES apres {n} etapes!"

        # DEMO_3
        if "quad_comm" in theorem:
            if n < 4:
                state_mgr.add_verification_result(attempt_id, False, f"{n}/4", "continue", "", 100.0)
                state_mgr.designate_next_agent("CriticAgent")
                return f"[VerifierAgent] Etape {n}/4 OK. -> CriticAgent"
            else:
                state_mgr.add_verification_result(attempt_id, True, "OK", "", "", 150.0)
                state_mgr.set_proof_complete(last.tactic)
                return f"[VerifierAgent] SUCCES apres {n} reecritures!"

        # DEMO_4
        if "distrib_both" in theorem:
            if n < 5:
                state_mgr.add_verification_result(attempt_id, False, f"{n}/5", "continue", "", 120.0)
                state_mgr.designate_next_agent("CriticAgent")
                return f"[VerifierAgent] Etape {n}/5. -> CriticAgent"
            else:
                state_mgr.add_verification_result(attempt_id, True, "OK", "", "", 200.0)
                state_mgr.set_proof_complete(last.tactic)
                return f"[VerifierAgent] SUCCES apres {n} etapes!"

        state_mgr.add_verification_result(attempt_id, False, "", "error", "", 50.0)
        state_mgr.designate_next_agent("CriticAgent")
        return "[VerifierAgent] Echec. -> CriticAgent"

    def _do_critic(self, state: ProofState, theorem: str) -> str:
        """Analyse critique."""
        state_mgr = self.plugins.get("state")
        if not state_mgr:
            return "[CriticAgent] Plugin manquant."

        n = len(state.tactics_history)

        if "n + m + 0 = m + n" in theorem:
            state_mgr.designate_next_agent("TacticAgent")
            return "[CriticAgent] add_zero OK, besoin add_comm. -> TacticAgent"

        if "quad_comm" in theorem:
            if n < 3:
                state_mgr.designate_next_agent("TacticAgent")
                return f"[CriticAgent] Continuer reecritures ({n}/4). -> TacticAgent"
            else:
                state_mgr.designate_next_agent("CoordinatorAgent")
                return "[CriticAgent] Besoin strategie globale. -> CoordinatorAgent"

        if "distrib_both" in theorem:
            if n < 4:
                state_mgr.designate_next_agent("TacticAgent")
                return f"[CriticAgent] Continuer decomposition ({n}/5). -> TacticAgent"
            else:
                state_mgr.designate_next_agent("CoordinatorAgent")
                return "[CriticAgent] Finaliser. -> CoordinatorAgent"

        state_mgr.designate_next_agent("TacticAgent")
        return "[CriticAgent] Reessayer. -> TacticAgent"

    def _do_coordinate(self, state: ProofState, theorem: str) -> str:
        """Coordination."""
        state_mgr = self.plugins.get("state")
        if not state_mgr:
            return "[CoordinatorAgent] Plugin manquant."

        if "quad_comm" in theorem:
            state_mgr.set_proof_strategy("ac_normalization")
            state_mgr.designate_next_agent("TacticAgent")
            return "[CoordinatorAgent] Strategie: AC normalization. -> TacticAgent"

        if "distrib_both" in theorem:
            state_mgr.set_proof_strategy("ring_solver")
            state_mgr.designate_next_agent("TacticAgent")
            return "[CoordinatorAgent] Strategie: ring solver. -> TacticAgent"

        state_mgr.designate_next_agent("TacticAgent")
        return "[CoordinatorAgent] Default. -> TacticAgent"


    def _call_llm(self, message: str, state: ProofState) -> str:
        """Appelle le LLM OpenAI avec function calling."""
        state_summary = json.dumps(state.get_state_snapshot(summarize=True), indent=2)
        tools = self._build_openai_tools()

        nl = chr(10)
        user_content = f"ETAT ACTUEL:{nl}{state_summary}{nl}{nl}TACHE:{nl}{message}"
        messages = [
            {"role": "system", "content": self.instructions},
            {"role": "user", "content": user_content}
        ]

        max_tool_calls = 10
        tool_results = []

        for iteration in range(max_tool_calls):
            try:
                model = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-5.2")
                use_mct = any(model.startswith(p) for p in ('gpt-4.5', 'gpt-5', 'o1', 'o3'))
                token_param = {"max_completion_tokens": 1000} if use_mct else {"max_tokens": 1000}

                response = self._openai_client.chat.completions.create(
                    model=model,
                    messages=messages,
                    tools=tools if tools else None,
                    tool_choice="auto" if tools else None,
                    temperature=0.3,
                    **token_param
                )

                assistant_message = response.choices[0].message

                if assistant_message.tool_calls:
                    messages.append(assistant_message.model_dump())

                    for tool_call in assistant_message.tool_calls:
                        func_name = tool_call.function.name
                        try:
                            arguments = json.loads(tool_call.function.arguments)
                        except json.JSONDecodeError:
                            arguments = {}

                        result = self._execute_tool_call(func_name, arguments)
                        tool_results.append(func_name.split("__")[-1])

                        messages.append({
                            "role": "tool",
                            "tool_call_id": tool_call.id,
                            "content": result
                        })
                else:
                    final_response = assistant_message.content or "(pas de reponse)"
                    if tool_results:
                        actions = ", ".join(tool_results[:5])
                        final_response = f"Actions: {actions}{nl}{final_response}"
                    return f"[{self.name}] {final_response}"

            except Exception as e:
                return f"[{self.name}] Erreur LLM: {e}"

        actions = ", ".join(tool_results[:5])
        return f"[{self.name}] Max tool calls. Actions: {actions}"


### 2.2. Patterns de Delegation Multi-Agents

Les instructions ci-dessus definissent les **regles de delegation** entre agents :

| Agent | Role | Delegue vers |
|-------|------|-------------|
| **SearchAgent** | Recherche lemmes Mathlib | TacticAgent (si lemmes trouves) |
| **TacticAgent** | Genere tactiques Lean | VerifierAgent (toujours) |
| **VerifierAgent** | Verifie preuve formelle | CriticAgent (si echec) / COMPLETE (si succes) |
| **CriticAgent** | Analyse erreurs | SearchAgent (retry) / CoordinatorAgent (si bloque) |
| **CoordinatorAgent** | Re-orchestre strategie | SearchAgent (nouvelle strategie) |

**Flow nominal** (preuve simple) :
```
SearchAgent → TacticAgent → VerifierAgent → COMPLETE
```

**Flow avec echec** (preuve complexe) :
```
SearchAgent → TacticAgent → VerifierAgent (FAIL)
   ↓
CriticAgent analyse erreur
   ↓
   +-- Erreur simple → SearchAgent (retry avec nouvelles contraintes)
   +-- Erreur complexe → CoordinatorAgent (changement strategie)
```

**Note critique** : Les demos actuelles (DEMO_1-3) sont trop triviales et ne declenchent JAMAIS CriticAgent ni CoordinatorAgent. DEMO_4 (list_length_append) devrait necessiter induction et potentiellement trigger ces agents.


### 2.3. Quand CriticAgent et CoordinatorAgent Interviennent

#### CriticAgent : Analyse d'Echecs de Tactiques

**Declenche par VerifierAgent quand** :
- `verify_proof()` retourne `success=False`
- Erreur Lean detectee : type mismatch, tactic failed, unknown identifier
- Preuve incomplete apres application de tactique

**Responsabilites** :
1. Parser l'erreur Lean (extraire type, message, contexte)
2. Identifier la cause (lemme incorrect, tactique inadequate, goal mal compris)
3. Proposer correction :
   - Erreur simple (lemme manquant) → Delegue SearchAgent avec contraintes
   - Erreur complexe (strategie incorrecte) → Delegue CoordinatorAgent

**Exemple d'intervention** :
```
[Tour 5] VerifierAgent: FAIL - "type mismatch, expected Nat but got Bool"
[Tour 6] CriticAgent: "TacticAgent a applique 'exact lemma_bool' mais goal attend Nat.
                       SearchAgent doit chercher lemmes avec type Nat -> Nat."
[Tour 7] SearchAgent: Recherche lemmes type-aware...
```

**Pourquoi absent des demos actuelles** :
- DEMO_1-3 : Lemmes Mathlib correspondent exactement au goal
- Pas de type mismatch, pas de tactic failure
- VerifierAgent retourne success au premier essai

#### CoordinatorAgent : Re-Orchestration Strategique

**Declenche par CriticAgent quand** :
- Echecs multiples consecutifs (3+ iterations sans progres)
- Strategie actuelle bloquee (EXPLORATION → REFINEMENT → toujours FAIL)
- Pattern d'erreur complexe (induction necessaire mais pas tentee)

**Responsabilites** :
1. Analyser historique complet (ProofState.snapshots)
2. Identifier pattern d'echec (loop, strategie inadequate)
3. Changer strategie globale :
   - EXPLORATION → VALIDATION (essayer preuves directes)
   - REFINEMENT → RECOVERY (backtrack + nouvelle approche)
4. Reset partiel de ProofState (clear failed tactics, keep lemmas)

**Exemple d'intervention** :
```
[Tour 8] CriticAgent: "Echec 3x consecutif avec meme lemme. Strategie bloquee."
[Tour 9] CoordinatorAgent: "Detection pattern: goal necessite induction mais pas tentee.
                            Changement strategie: EXPLORATION → RECOVERY.
                            Ajout contrainte: TacticAgent DOIT considerer 'induction'."
[Tour 10] SearchAgent: Recherche lemmes inductifs...
```

**Pourquoi absent des demos actuelles** :
- DEMO_1-3 : Pas d'echecs, donc CriticAgent jamais declenche
- DEMO_4 (list_length_append) : **DEVRAIT** declencher si :
  - Lemme direct `List.length_append` pas trouve
  - TacticAgent essaie `rw` ou `simp` sans induction → echec
  - CriticAgent detecte besoin d'induction
  - CoordinatorAgent change strategie vers RECOVERY

#### Activation des Agents Critiques

| Scenario | SearchAgent | TacticAgent | VerifierAgent | CriticAgent | CoordinatorAgent |
|----------|-------------|-------------|---------------|-------------|------------------|
| **Preuve triviale** (rfl) | ✗ | ✓ | ✓ | ✗ | ✗ |
| **Lemme direct trouve** (exact) | ✓ | ✓ | ✓ | ✗ | ✗ |
| **Lemme incorrect** (type mismatch) | ✓ | ✓ | ✓ | ✓ | ✗ |
| **Tactique echoue 1x** (retry) | ✓ | ✓ | ✓ | ✓ | ✗ |
| **Tactique echoue 3x** (bloque) | ✓ | ✓ | ✓ | ✓ | ✓ |
| **Induction necessaire** | ✓ | ✓ | ✓ | ✓ | ✓ |

**Conclusion** : Pour tester CriticAgent et CoordinatorAgent, nous devons utiliser des theoremes ou :
1. Mathlib n'a PAS de lemme direct exact match
2. Preuve necessite composition de tactiques (rw + simp + induction)
3. Premiere tentative echoue et necessite correction

**DEMO_4 (list_length_append) est concu pour ca** - mais seulement si on desactive l'acces au lemme `List.length_append` de Mathlib.


In [23]:


# =============================================================================
# Factory pour creer les agents (SK ou fallback)
# =============================================================================

def create_agents(
    plugins: Dict[str, Any],
    state: ProofState,
    use_sk: bool = True,
    use_simulation: bool = False
) -> Dict[str, Any]:
    """
    Cree les 5 agents specialises.

    Args:
        plugins: Dictionnaire des plugins SK
        state: Etat partage de la preuve
        use_sk: Utiliser Semantic Kernel si disponible
        use_simulation: Mode simulation (sans appels LLM)

    Returns:
        Dictionnaire {nom_agent: agent}
    """
    if use_sk and SK_AVAILABLE and not use_simulation:
        return _create_sk_agents(plugins, state)
    else:
        return _create_simple_agents(plugins, use_simulation)


def _create_simple_agents(plugins: Dict[str, Any], use_simulation: bool) -> Dict[str, Any]:
    """Cree les agents en mode fallback/simulation."""
    return {
        "SearchAgent": SimpleAgent("SearchAgent", SEARCH_AGENT_INSTRUCTIONS, plugins, use_simulation),
        "TacticAgent": SimpleAgent("TacticAgent", TACTIC_AGENT_INSTRUCTIONS, plugins, use_simulation),
        "VerifierAgent": SimpleAgent("VerifierAgent", VERIFIER_AGENT_INSTRUCTIONS, plugins, use_simulation),
        "CriticAgent": SimpleAgent("CriticAgent", CRITIC_AGENT_INSTRUCTIONS, plugins, use_simulation),
        "CoordinatorAgent": SimpleAgent("CoordinatorAgent", COORDINATOR_AGENT_INSTRUCTIONS, plugins, use_simulation),
    }


def _create_sk_agents(plugins: Dict[str, Any], state: ProofState) -> Dict[str, Any]:
    """
    Cree les agents avec Semantic Kernel ChatCompletionAgent.

    Utilise:
    - OpenAIChatCompletion pour le service LLM
    - FunctionChoiceBehavior.Auto() pour le function calling automatique
    - Les plugins existants sont passes aux agents
    """
    # Creer le kernel et le service
    kernel = Kernel()
    api_key = os.getenv("OPENAI_API_KEY")
    model = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-5.2")

    service = OpenAIChatCompletion(
        service_id="openai",
        ai_model_id=model,
        api_key=api_key
    )
    kernel.add_service(service)

    # Ajouter les plugins au kernel
    for plugin_name, plugin in plugins.items():
        kernel.add_plugin(plugin, plugin_name=plugin_name)

    # Configuration pour auto function calling
    settings = kernel.get_prompt_execution_settings_from_service_id(service_id="openai")
    settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

    # Creer les agents
    agents = {}
    agent_configs = [
        ("SearchAgent", SEARCH_AGENT_INSTRUCTIONS),
        ("TacticAgent", TACTIC_AGENT_INSTRUCTIONS),
        ("VerifierAgent", VERIFIER_AGENT_INSTRUCTIONS),
        ("CriticAgent", CRITIC_AGENT_INSTRUCTIONS),
        ("CoordinatorAgent", COORDINATOR_AGENT_INSTRUCTIONS),
    ]

    for name, instructions in agent_configs:
        agents[name] = ChatCompletionAgent(
            kernel=kernel,
            name=name,
            instructions=instructions,
            arguments=KernelArguments(settings=settings)
        )

    print(f"Crees {len(agents)} agents SK avec modele {model}")
    return agents


# =============================================================================
# Test des Agents
# =============================================================================

print("\n=== Test des Agents ===")

# Creer l'environnement
test_state = ProofState(
    theorem_statement="theorem add_zero (n : Nat) : n + 0 = n",
    current_goal="n + 0 = n"
)
runner = LeanRunner(backend="subprocess", timeout=30)

# Creer les plugins
plugins = {
    "state": ProofStateManagerPlugin(test_state),
    "search": LeanSearchPlugin(runner),
    "tactic": LeanTacticPlugin(),
    "verification": LeanVerificationPlugin(runner)
}

# Mode de fonctionnement
USE_SK = SK_AVAILABLE and os.getenv("OPENAI_API_KEY")
USE_SIMULATION = not USE_SK  # Simulation si SK non disponible ou pas de cle API

print(f"Mode: {'Semantic Kernel' if USE_SK else 'Simulation'}")

# Creer les agents
agents = create_agents(plugins, test_state, use_sk=USE_SK, use_simulation=USE_SIMULATION)

# Test rapide en mode simulation
if USE_SIMULATION:
    print("\nTest SearchAgent (simulation):")
    response = agents["SearchAgent"].invoke("Trouve des lemmes pour n + 0 = n", test_state)
    print(response)
    print(f"Etat apres SearchAgent:\n{test_state}")



=== Test des Agents ===
Mode: Semantic Kernel
Crees 5 agents SK avec modele gpt-5.2


### 2.4. Vue d'Ensemble des 5 Agents Specialises

La fonction `create_agents()` instancie les 5 agents avec :
- **Instructions** : Prompts systemiques definissant role et regles de delegation
- **Plugins** : Fonctions exposees (search, tactic generation, verification, etc.)
- **Modele LLM** : gpt-5.2 (ou simulation si mode LLM desactive)

#### Signatures des agents

```python
SearchAgent(
    plugins=[lean_search_plugin, state_plugin],
    instructions="Trouve lemmes Mathlib pertinents..."
)

TacticAgent(
    plugins=[tactic_plugin, state_plugin],
    instructions="Genere tactiques Lean avec confiance..."
)

VerifierAgent(
    plugins=[verification_plugin, state_plugin],
    instructions="Compile et verifie preuves formelles..."
)

CriticAgent(
    plugins=[state_plugin],
    instructions="Analyse echecs et propose corrections..."
)

CoordinatorAgent(
    plugins=[state_plugin],
    instructions="Re-orchestre strategie globale..."
)
```

**Pattern cle** : Chaque agent n'a acces qu'aux plugins dont il a besoin (principe de moindre privilege). Le `state_plugin` est partage par tous pour consulter/modifier ProofState.


## 3. Orchestration Multi-Agents

L'orchestration determine comment les agents sont selectionnes et quand la conversation se termine.

**DelegatingSelectionStrategy** (Pattern recommande):
- Chaque agent designe explicitement le suivant via `designate_next_agent()`
- Si aucune designation, utilise un agent par defaut (CoordinatorAgent)

**ProofCompleteTermination**:
- Termine si `proof_complete == True`
- Termine si `iteration_count >= max_iterations`

### 3.1. Workflow Complet de Preuve

Cette demonstration montre le workflow multi-agents complet:
1. **CoordinatorAgent** initialise la session
2. **SearchAgent** recherche les lemmes pertinents
3. **TacticAgent** propose des tactiques
4. **VerifierAgent** verifie avec Lean
5. **CriticAgent** intervient en cas d'echec

## 🎭 Orchestration Multi-Agents

### Le problème de l'orchestration

Avec 5 agents, qui parle quand ? Deux approches :

1. **Statique** : SearchAgent → TacticAgent → VerifierAgent (toujours)
   - Simple mais rigide
   - Pas de backtracking

2. **Dynamique** : Décisions basées sur l'état de la preuve
   - Flexible mais complexe
   - Permet le backtracking et la récupération d'erreur

**Nous utilisons l'approche dynamique.**

### Stratégies d'orchestration

#### ProofSelectionStrategy

Décide **quel agent agit** à chaque tour :

```python
class ProofSelectionStrategy:
    def select_next_agent(self, state: ProofState, agents: List[str]) -> str:
        if state.phase == ProofPhase.INIT:
            return "CoordinatorAgent"
        elif state.phase == ProofPhase.SEARCH:
            return "SearchAgent"
        # ...
```

#### ProofTerminationStrategy

Décide **quand arrêter** la session :

```python
class ProofTerminationStrategy:
    def should_terminate(self, state: ProofState, iteration: int) -> Tuple[bool, str]:
        if state.phase == ProofPhase.COMPLETE:
            return (True, "Preuve complète!")
        if iteration >= max_iterations:
            return (True, "Timeout atteint")
        # ...
```

### Boucle principale

```python
while not should_terminate:
    # 1. Sélectionner agent
    agent_name = selection_strategy.select_next_agent(state, agents)

    # 2. Exécuter agent (appelle le LLM)
    response = agent.chat(f"Phase: {state.phase}, Goal: {state.current_goal}")

    # 3. L'agent appelle des plugins (modifie l'état)
    # Exemple: log_tactic_attempt("rw [Nat.add_zero]")

    # 4. Mettre à jour phase selon résultat
    update_phase(state)

    # 5. Vérifier condition de terminaison
    should_terminate, reason = termination_strategy.should_terminate(state, iteration)
```

### Snapshots : Observer l'orchestration

À chaque tour, on sauvegarde :

```json
{
  "iteration": 5,
  "agent": "TacticAgent",
  "phase_before": "SEARCH",
  "phase_after": "TACTIC_GEN",
  "action": "Generated tactic: rw [Nat.add_zero]",
  "state_snapshot": {...}
}
```

**Utilité** : Voir exactement quelle décision chaque agent a prise.

In [24]:
# =============================================================================
# Section 8.7 - Strategies d'Orchestration (Pattern Argument_Analysis)
# =============================================================================
# Strategies personnalisees basees sur l'etat partage (pas sur l'historique)

# Fix for Jupyter event loop
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    pass

import logging
from typing import Dict, Any, List, Optional


### 3.2. ProofSelectionStrategy : Selection d'Agent : Selection d'Agent Basee sur l'Etat

**Pattern inspire de Argument_Analysis** : Selection d'agent via **designation explicite** dans ProofState.

#### Architecture

```python
class ProofSelectionStrategy(SelectionStrategy):
    async def next(agents, history) -> Agent:
        # 1. Lire designation explicite
        designated = state.consume_next_agent_designation()
        
        # 2. Si designation presente, utiliser cet agent
        if designated:
            return agents_map[designated]
        
        # 3. Sinon, utiliser agent par defaut (SearchAgent)
        return agents_map[default_agent_name]
```

#### Difference avec Semantic Kernel Standard

| Semantic Kernel Standard | ProofSelectionStrategy (Custom) |
|-------------------------|--------------------------------|
| Parse historique messages | Lit `state.next_agent_designation` |
| Selection basee sur keywords | Selection basee sur etat partage |
| Stateless (pas de memoire) | Stateful (ProofState) |
| Complexite O(n) messages | Complexite O(1) |

**Avantage** : Chaque agent designe explicitement son successeur via `state.designate_next_agent()`, evitant parsing d'historique fragile.


In [25]:
# =============================================================================
# ProofSelectionStrategy - Selection basee sur l'etat partage
# =============================================================================
# NOTE: Ces classes SK ne sont definies que si SK est disponible.
# Le mode simulation n'en a pas besoin.

if SK_AVAILABLE:
    from semantic_kernel.agents.strategies.selection.selection_strategy import SelectionStrategy

    class ProofSelectionStrategy(SelectionStrategy):
        """Strategie de selection SK (non utilisee en mode simulation)."""
        pass
else:
    print("ProofSelectionStrategy: Skipped (SK non disponible)")


### 3.3. ProofTerminationStrategy : Detection de Completion : Detection de Completion

**Responsabilite** : Detecter quand arreter l'orchestration multi-agents.

#### Criteres de Terminaison

```python
class ProofTerminationStrategy(TerminationStrategy):
    async def should_terminate(agents, history) -> bool:
        # 1. Preuve complete detectee
        if state.proof_complete:
            return True
        
        # 2. Max iterations atteint
        if state.current_iteration >= max_iterations:
            return True
        
        # 3. Timeout (optionnel)
        if time.time() - start_time > timeout:
            return True
        
        return False
```

#### Comparaison avec Autres Patterns

| Pattern | Terminaison basee sur | Avantages | Inconvenients |
|---------|----------------------|-----------|---------------|
| **Message-based** | Keyword dans dernier message ("DONE", "COMPLETE") | Simple, standard SK | Fragile, depend du LLM |
| **State-based** (ce notebook) | `state.proof_complete` flag | Robuste, deterministe | Necessite etat partage |
| **Iteration-based** | Compteur max iterations | Toujours termine | Peut stopper preuve incomplete |
| **Consensus-based** | Vote agents (majorite) | Robuste aux erreurs | Complexe, lent |

**Notre choix** : Combinaison **state-based + iteration-based** pour garantir terminaison.


In [26]:
# =============================================================================
# ProofTerminationStrategy - Terminaison basee sur l'etat partage
# =============================================================================
# NOTE: Ces classes SK ne sont definies que si SK est disponible.
# Le mode simulation n'en a pas besoin.

if SK_AVAILABLE:
    from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy

    class ProofTerminationStrategy(TerminationStrategy):
        """Strategie de terminaison SK (non utilisee en mode simulation)."""
        pass
else:
    print("ProofTerminationStrategy: Skipped (SK non disponible)")


### 3.4. ProofAgentGroupChat : Chat Multi-Agents : Orchestration Multi-Agents

**Classe orchestrateur** pour gérer la conversation multi-agents avec Semantic Kernel.

#### Architecture

```python
class ProofAgentGroupChat:
    def __init__(agents, state, use_sk=True):
        self.agents = agents  # Dict[str, ChatCompletionAgent]
        self.state = state    # ProofState partage
    
    def run(initial_message, verbose=True) -> str:
        # Execute conversation multi-agents
        if use_sk:
            return await _run_sk(...)  # Semantic Kernel
        else:
            return _run_fallback(...)   # Simulation
```

**Pattern clé** :
- Utilise `ProofSelectionStrategy` pour sélectionner agents
- Utilise `ProofTerminationStrategy` pour détecter fin
- Crée `AgentGroupChat` de Semantic Kernel avec ces stratégies
- Fallback en mode simulation si SK non disponible


In [27]:
class ProofAgentGroupChat:
    """
    Orchestre les agents pour la preuve de theoremes.
    Supporte mode simulation (SimpleAgent) et mode SK (ChatCompletionAgent).
    """

    def __init__(self, agents: Dict[str, Any], state: ProofState, use_sk: bool = True):
        self.agents = agents
        self.state = state
        self.use_sk = use_sk and SK_AVAILABLE
        self.history = []
        self._proof_tactics_found = []  # Track tactics found across iterations

    def run(self, initial_message: str, verbose: bool = True) -> str:
        """Execute la conversation multi-agents."""
        if self.use_sk:
            # Mode Semantic Kernel - utilise async
            import asyncio
            import nest_asyncio
            nest_asyncio.apply()
            try:
                loop = asyncio.get_event_loop()
                return loop.run_until_complete(self._run_sk(initial_message, verbose))
            except RuntimeError:
                return asyncio.run(self._run_sk(initial_message, verbose))
        else:
            # Mode simulation - sync
            return self._run_fallback(initial_message, verbose)

    async def _run_sk(self, initial_message: str, verbose: bool = True) -> str:
        """Execution avec Semantic Kernel ChatCompletionAgent."""
        from semantic_kernel.contents.chat_history import ChatHistory
        from semantic_kernel.contents.chat_message_content import ChatMessageContent
        from semantic_kernel.contents.utils.author_role import AuthorRole

        if verbose:
            print("=" * 60)
            print(f"Session MULTI-AGENTS (SK): {initial_message[:80]}...")
            print("=" * 60)

        # Creer l'historique de chat partage entre les agents
        chat_history = ChatHistory()
        chat_history.add_user_message(initial_message)

        current_message = initial_message
        agent_order = ["SearchAgent", "TacticAgent", "VerifierAgent", "CriticAgent", "CoordinatorAgent"]

        for i in range(self.state.max_iterations):
            self.state.iteration = i + 1
            self.state.increment_iteration()

            # Determiner l'agent a utiliser
            designated = self.state.consume_next_agent_designation()
            if designated and designated in self.agents:
                agent_name = designated
            else:
                agent_name = agent_order[i % len(agent_order)]

            agent = self.agents.get(agent_name)
            if not agent:
                continue

            if verbose:
                print(f"\n[Tour {self.state.iteration_count}] {agent_name}")

            # Invoquer l'agent SK de maniere asynchrone
            try:
                response_text = ""

                # ChatCompletionAgent.invoke() prend un ChatHistory et retourne un AsyncIterable
                async for response in agent.invoke(chat_history):
                    if hasattr(response, 'content') and response.content:
                        response_text += str(response.content)
                    elif hasattr(response, 'items'):
                        # Si c'est un ChatMessageContent avec items
                        for item in response.items:
                            if hasattr(item, 'text'):
                                response_text += item.text

                if not response_text:
                    response_text = f"[{agent_name}] Pas de reponse"

                # Ajouter la reponse a l'historique
                chat_history.add_assistant_message(response_text)

                # Ajouter le prochain message utilisateur (contexte pour le prochain agent)
                if i < self.state.max_iterations - 1:
                    next_context = f"Continue la preuve. Reponse precedente de {agent_name}: {response_text[:200]}"
                    chat_history.add_user_message(next_context)

                # Mettre a jour l'etat selon la reponse
                self._update_state_from_response(agent_name, response_text)

            except Exception as e:
                import traceback
                response_text = f"Erreur agent {agent_name}: {str(e)}"
                if verbose:
                    print(f"  [ERROR] {e}")
                    traceback.print_exc()

            self.history.append({
                "iteration": self.state.iteration_count,
                "agent": agent_name,
                "response": response_text[:200] if response_text else "",
            })

            if verbose:
                display_response = response_text[:200] + "..." if len(response_text) > 200 else response_text
                print(f"  Response: {display_response}")

            if self.state.proof_complete:
                if verbose:
                    print(f"\n[LOG] Preuve trouvee!")
                break

            current_message = response_text

        if verbose:
            print("\n" + "=" * 60)
            print(f"Session terminee apres {self.state.iteration_count} iterations.")
            print("=" * 60)

        return self.state.final_proof or "Preuve non trouvee"

    def _update_state_from_response(self, agent_name: str, response: str):
        """Met a jour l'etat partage en fonction de la reponse de l'agent."""
        import re
        response_lower = response.lower()

        # Detection des lemmes decouverts
        if "lemma:" in response_lower or "found:" in response_lower or "nat." in response_lower:
            lemma_matches = re.findall(r'(Nat\.\w+|Eq\.\w+|List\.\w+)', response)
            for lemma in lemma_matches:
                if lemma not in self.state.discovered_lemmas:
                    self.state.discovered_lemmas.append(lemma)

        # Detection des tactiques - track across iterations
        proof_patterns = [
            (r'simp\s*\[[^\]]*\]', 'simp'),
            (r'\brfl\b', 'rfl'),
            (r'exact\s+\w+', 'exact'),
            (r'\bring\b', 'ring'),
            (r'\bomega\b', 'omega'),
            (r'\blinarith\b', 'linarith'),
            (r'\bdecide\b', 'decide'),
        ]

        for pattern, tactic_name in proof_patterns:
            if re.search(pattern, response, re.IGNORECASE):
                if tactic_name not in self._proof_tactics_found:
                    self._proof_tactics_found.append(tactic_name)
                    self.state.tactics_history.append(response[:100])

        # Detection de preuve complete - multiple signals
        proof_complete_signals = [
            "proof complete",
            "qed",
            "verified",
            "goals accomplished",
            "proof found",
            "la preuve est terminee",
            "la preuve est cloturee",
            "preuve reussie",
        ]

        if any(signal in response_lower for signal in proof_complete_signals):
            # If we have found tactics earlier, mark as complete
            if self._proof_tactics_found:
                self.state.phase = ProofPhase.COMPLETE
                if not self.state.final_proof:
                    self.state.final_proof = self._proof_tactics_found[0]
        elif ":= by" in response and self._proof_tactics_found:
            # Lean-style proof block detected with tactics
            self.state.phase = ProofPhase.COMPLETE
            if not self.state.final_proof:
                self.state.final_proof = self._proof_tactics_found[0]

        # Alternative: detect complete proof in code block
        code_block_match = re.search(r'```lean\n(.*?)```', response, re.DOTALL)
        if code_block_match:
            code_content = code_block_match.group(1)
            if ":= by" in code_content or ":= rfl" in code_content:
                # Check for proof tactics in the code block
                for pattern, tactic_name in proof_patterns:
                    if re.search(pattern, code_content, re.IGNORECASE):
                        self.state.phase = ProofPhase.COMPLETE
                        self.state.final_proof = code_content.strip()[:200]
                        break

        # Detection de delegation
        delegate_patterns = [
            (r'@TacticAgent|delegate.*TacticAgent', 'TacticAgent'),
            (r'@VerifierAgent|delegate.*VerifierAgent', 'VerifierAgent'),
            (r'@CriticAgent|delegate.*CriticAgent', 'CriticAgent'),
            (r'@CoordinatorAgent|delegate.*CoordinatorAgent', 'CoordinatorAgent'),
            (r'@SearchAgent|delegate.*SearchAgent', 'SearchAgent'),
        ]
        for pattern, target in delegate_patterns:
            if re.search(pattern, response, re.IGNORECASE):
                self.state.designate_next_agent(target)
                break

    def _run_fallback(self, initial_message: str, verbose: bool = True) -> str:
        """Execution sans Semantic Kernel (mode simulation)."""
        if verbose:
            print("=" * 60)
            print(f"Session MULTI-AGENTS: {initial_message[:80]}...")
            print("=" * 60)

        current_message = initial_message
        agent_order = ["SearchAgent", "TacticAgent", "VerifierAgent", "CriticAgent", "CoordinatorAgent"]

        for i in range(self.state.max_iterations):
            self.state.iteration = i + 1

            designated = self.state.consume_next_agent_designation()
            if designated and designated in self.agents:
                agent_name = designated
            else:
                agent_name = agent_order[i % len(agent_order)]

            agent = self.agents.get(agent_name)
            if not agent:
                continue

            if verbose:
                print(f"\n[Tour {self.state.iteration_count}] {agent_name}")

            response = agent.invoke(current_message, self.state)

            self.history.append({
                "iteration": self.state.iteration_count,
                "agent": agent_name,
                "response": response[:200] if response else "",
            })

            if verbose:
                print(f"  Response: {response[:200]}..." if len(response) > 200 else f"  Response: {response}")

            if self.state.proof_complete:
                if verbose:
                    print(f"\n[LOG] Preuve trouvee!")
                break

            current_message = response

        if verbose:
            print("\n" + "=" * 60)
            print(f"Session terminee apres {self.state.iteration_count} iterations.")
            print("=" * 60)

        return self.state.final_proof or "Preuve non trouvee"


# Test rapide
print("ProofAgentGroupChat defini (mode SK + simulation)")

ProofAgentGroupChat defini (mode SK + simulation)


### 3.5. Test des Strategies

Code de test pour valider :
- **ProofTerminationStrategy** : Détecte `state.proof_complete`
- **SimpleOrchestratorAgent** : Exécute conversation avec désignation d'agents

**Exécution automatique** lors du chargement de la cellule.


In [28]:
# Test des Strategies
# =============================================================================

print("=== Test des Strategies ===")

test_state = ProofState(
    theorem_statement="theorem test (n : Nat) : n = n",
    current_goal="n = n",
    max_iterations=5
)

print(f"State cree: {test_state.session_id}")
print(f"Phase initiale: {test_state.phase.value}")

# Test designation
test_state.designate_next_agent("TacticAgent")
designated = test_state.consume_next_agent_designation()
print(f"Designation test: {designated}")

# Test proof_complete
print(f"proof_complete initial: {test_state.proof_complete}")
test_state.phase = ProofPhase.COMPLETE
print(f"proof_complete apres COMPLETE: {test_state.proof_complete}")

print("\nStrategies pretes pour utilisation avec AgentGroupChat")


=== Test des Strategies ===
State cree: 322a0c82
Phase initiale: init
Designation test: TacticAgent
proof_complete initial: False
proof_complete apres COMPLETE: True

Strategies pretes pour utilisation avec AgentGroupChat


### Les 4 demos avec progression reelle

Les theoremes sont choisis pour necessiter des **strategies de preuve differentes**, pas juste `simp`.

#### 1 DEMO_1_TRIVIAL : `theorem demo_rfl (n : Nat) : n = n`

- **Complexite** : Triviale
- **Strategie** : `rfl` (reflexivite)
- **Iterations attendues** : 1-2
- **But** : Validation du pipeline basique

#### 2 DEMO_2_DEFINITION : `theorem zero_add (n : Nat) : 0 + n = n`

- **Complexite** : Simple
- **Strategie** : `simp [Nat.zero_add]` ou induction
- **Iterations attendues** : 2-4
- **Piege** : `simp` seul ne suffit pas toujours, peut necessiter `Nat.zero_add`
- **But** : Test de recherche de lemmes specifiques

#### 3 DEMO_3_INDUCTION : `theorem add_assoc_manual (a b c : Nat) : (a + b) + c = a + (b + c)`

- **Complexite** : Intermediaire
- **Strategie** : Induction structuree sur `a`
- **Iterations attendues** : 3-6
- **Structure** :
  ```lean
  induction a with
  | zero => simp
  | succ a ih => simp [Nat.succ_add, ih]
  ```
- **But** : Test de generation de preuves par induction

#### 4 DEMO_4_COMPOSED : `theorem mul_comm_manual (m n : Nat) : m * n = n * m`

- **Complexite** : Avancee
- **Strategie** : Induction + lemmes auxiliaires (mul_succ, succ_mul, add_comm)
- **Iterations attendues** : 5-10
- **Echecs possibles** : Premiere tentative peut echouer, necessite CriticAgent
- **But** : Stress-test du systeme multi-agents complet

In [29]:
# =============================================================================
# Section 8.8 - Demonstration Complete
# =============================================================================

def prove_with_multi_agents(
    theorem: str,
    goal: str = "",
    max_iterations: int = 20,
    verbose: bool = True,
    use_simulation: bool = None  # None = auto-detect
) -> Dict[str, Any]:
    """
    Prouve un theoreme en utilisant le systeme multi-agents.

    Args:
        theorem: L'enonce du theoreme complet
        goal: Le but a prouver (extrait du theoreme si non fourni)
        max_iterations: Nombre maximum d'iterations
        verbose: Afficher les logs
        use_simulation: True=simulation, False=LLM reel, None=auto

    Returns:
        Dict avec resultats et metriques
    """
    import time
    start_time = time.time()

    # Auto-detection du mode
    if use_simulation is None:
        api_key = os.getenv("OPENAI_API_KEY", "")
        has_valid_key = api_key and len(api_key) > 10 and not api_key.startswith("sk-...")
        use_simulation = not has_valid_key

    # 1. Creer l'etat
    if not goal:
        if ":" in theorem:
            goal = theorem.split(":")[-1].strip()

    state = ProofState(
        theorem_statement=theorem,
        current_goal=goal,
        max_iterations=max_iterations
    )

    # 2. Creer le runner Lean
    runner = LeanRunner(backend="subprocess", timeout=30)

    # 3. Creer les plugins
    plugins = {
        "state": ProofStateManagerPlugin(state),
        "search": LeanSearchPlugin(runner),
        "tactic": LeanTacticPlugin(),
        "verification": LeanVerificationPlugin(runner)
    }

    # 4. Creer les agents
    use_sk = SK_AVAILABLE and not use_simulation
    agents = create_agents(plugins, state, use_sk=use_sk, use_simulation=use_simulation)

    # 5. Configurer les strategies
    # Strategies gerees automatiquement par ProofAgentGroupChat

    # 6. Creer le groupe de chat
    chat = ProofAgentGroupChat(
        agents=agents,
        state=state,
        use_sk=use_sk
    )

    mode_str = "Semantic Kernel" if use_sk else ("Simulation" if use_simulation else "OpenAI direct")
    if verbose:
        print(f"Mode: {mode_str}")

    # 7. Executer
    result = chat.run(f"Prouver: {theorem}", verbose=verbose)

    # 8. Collecter les metriques
    elapsed = time.time() - start_time
    metrics = {
        "success": state.proof_complete,
        "theorem": theorem,
        "final_proof": state.final_proof,
        "iterations": state.iteration_count,
        "lemmas_discovered": len(state.discovered_lemmas),
        "tactics_tried": len(state.tactics_history),
        "verifications": len(state.verification_results),
        "total_time_s": round(elapsed, 2),
        "lean_time_ms": round(state.total_lean_time_ms, 2),
        "mode": mode_str
    }

    return metrics


# =============================================================================
# Test de la demonstration
# =============================================================================

print("\n" + "=" * 60)
print("DEMONSTRATION MULTI-AGENTS POUR THEOREM PROVING")
print("=" * 60)

# =============================================================================
# Section 8.8 - Démonstrations Progressives Multi-Agents
# =============================================================================

# Configuration
USE_LLM_MODE = True  # False pour simulation (defaut), True pour LLM reel

# Quatre théorèmes de complexité croissante
DEMOS = [
    {
        "name": "DEMO_1_TRIVIAL",
        "theorem": "theorem demo_rfl (n : Nat) : n = n",
        "description": "Reflexivite pure - rfl immediat",
        "expected_iterations": "1-2",
        "expected_lemmas": 1,
        "complexity": "Triviale",
        "strategy": "rfl"
    },
    {
        "name": "DEMO_2_DEFINITION",
        "theorem": "theorem zero_add (n : Nat) : 0 + n = n",
        "description": "Definition - necessite lemme Nat.zero_add",
        "expected_iterations": "1-2",
        "expected_lemmas": 1,
        "complexity": "Simple",
        "strategy": "exact Nat.zero_add n"
    },
    {
        "name": "DEMO_3_INDUCTION",
        "theorem": "theorem add_assoc_manual (a b c : Nat) : (a + b) + c = a + (b + c)",
        "description": "Associativite - peut utiliser lemme ou induction",
        "expected_iterations": "2-5",
        "expected_lemmas": 2,
        "complexity": "Intermediaire",
        "strategy": "induction ou Nat.add_assoc"
    },
    {
        "name": "DEMO_4_COMPOSED",
        "theorem": "theorem length_append_manual (xs ys : List Nat) : (xs ++ ys).length = xs.length + ys.length",
        "description": "Liste - induction sur structure de donnees",
        "expected_iterations": "3-6",
        "expected_lemmas": 3,
        "complexity": "Avancee",
        "strategy": "induction xs; simp"
    }
]

print("=" * 70)
print("DÉMONSTRATIONS PROGRESSIVES - SYSTÈME MULTI-AGENTS")
print("=" * 70)
print()

# Exécuter chaque démo
results_comparison = []




DEMONSTRATION MULTI-AGENTS POUR THEOREM PROVING
DÉMONSTRATIONS PROGRESSIVES - SYSTÈME MULTI-AGENTS



### 4.1. DEMO_1 : Preuve Triviale

**Objectif** : Valider le pipeline complet avec un theoreme trivial

**Theoreme** : `theorem demo_rfl (n : Nat) : n = n`

**Attentes** :
- **Iterations** : 1-2 (reflexivite immediate)
- **Agents impliques** : TacticAgent (rfl) → VerifierAgent
- **CriticAgent/CoordinatorAgent** : NON (preuve triviale)
- **Temps** : <1 seconde

Cette demo sert de **baseline** pour verifier que le systeme fonctionne.


In [30]:

# Execute DEMO_1
demo = DEMOS[0]
print("\n" + "=" * 70)
print(f"DEMO 1/4: {demo['name']}")
print("=" * 70)
print(f"Theoreme: {demo['theorem']}")
print(f"Complexite: {demo['complexity']}")
print(f"Iterations attendues: {demo['expected_iterations']}")
print(f"Lemmes necessaires: {demo['expected_lemmas']}")
print("=" * 70)

result_1 = prove_with_multi_agents(
    theorem=demo["theorem"],
    max_iterations=20,
    verbose=True,
    use_simulation=not USE_LLM_MODE
)

print(f"\nResultat DEMO_1:")
print(f"  - Success: {result_1['success']}")
print(f"  - Iterations: {result_1['iterations']}")
print(f"  - Proof: {result_1['final_proof'][:100] if result_1['final_proof'] else 'None'}...")



DEMO 1/4: DEMO_1_TRIVIAL
Theoreme: theorem demo_rfl (n : Nat) : n = n
Complexite: Triviale
Iterations attendues: 2-3
Lemmes necessaires: 1
Crees 5 agents SK avec modele gpt-5.2
Mode: Semantic Kernel
Session MULTI-AGENTS (SK): Prouver: theorem demo_rfl (n : Nat) : n = n...

[Tour 2] SearchAgent
  Response: Je délègue à TacticAgent avec les lemmes `rfl` et `Eq.refl`.
Delegation to TacticAgent.
Lemmes pertinents (Mathlib/Init) pour `theorem demo_rfl (n : Nat) : n = n` :

1. `rfl : a = a` (le plus direct)
...

[Tour 3] TacticAgent
  Response: Tactique proposée (confiance 0.99) : `rfl`

Preuve Lean complète :

```lean
theorem demo_rfl (n : Nat) : n = n := by
  rfl
```

Je délègue à **VerifierAgent** pour vérifier.

[LOG] Preuve trouvee!

Session terminee apres 3 iterations.

Resultat DEMO_1:
  - Success: True
  - Iterations: 3
  - Proof: theorem demo_rfl (n : Nat) : n = n := by
  rfl...


### 4.2. DEMO_2 : Composition de Lemmes

**Objectif** : Tester recherche + composition de 2 lemmes

**Theoreme** : `theorem add_zero_comm (n m : Nat) : n + m + 0 = m + n`

**Attentes** :
- **Iterations** : 5-7 (2 etapes de preuve)
- **Agents impliques** : SearchAgent → TacticAgent → VerifierAgent → CriticAgent → TacticAgent
- **Lemmes Mathlib attendus** : `Nat.add_zero`, `Nat.add_comm`
- **CriticAgent** : POSSIBLE (si premiere tactique incomplete)
- **Temps** : 1-3 secondes

Cette demo teste la **composition de lemmes** : d'abord eliminer le `+ 0`, puis appliquer la commutativite.

In [31]:

# Execute DEMO_2
demo = DEMOS[1]
print("\n" + "=" * 70)
print(f"DEMO 2/4: {demo['name']}")
print("=" * 70)
print(f"Theoreme: {demo['theorem']}")
print(f"Complexite: {demo['complexity']}")
print(f"Iterations attendues: {demo['expected_iterations']}")
print(f"Lemmes necessaires: {demo['expected_lemmas']}")
print("=" * 70)

result_2 = prove_with_multi_agents(
    theorem=demo["theorem"],
    max_iterations=20,
    verbose=True,
    use_simulation=not USE_LLM_MODE
)

print(f"\nResultat DEMO_2:")
print(f"  - Success: {result_2['success']}")
print(f"  - Iterations: {result_2['iterations']}")
print(f"  - Proof: {result_2['final_proof'][:100] if result_2['final_proof'] else 'None'}...")



DEMO 2/4: DEMO_2_COMPOSITION
Theoreme: theorem add_zero_comm (n m : Nat) : n + m + 0 = m + n
Complexite: Simple - necessite combiner 2 lemmes
Iterations attendues: 5-7
Lemmes necessaires: 2
Crees 5 agents SK avec modele gpt-5.2
Mode: Semantic Kernel
Session MULTI-AGENTS (SK): Prouver: theorem add_zero_comm (n m : Nat) : n + m + 0 = m + n...

[Tour 2] SearchAgent
  Response: 







































































































































































































...

[Tour 3] TacticAgent
  Response: {"to":"functions.meta-delegate_to_verifier_agent","task_summary":"Vérifier la preuve Lean. Tentative principale: `by simpa [Nat.add_assoc] using congrArg (fun t => t + 0) (Nat.add_comm n m)` (devrait ...

[Tour 4] VerifierAgent
  Response: La preuve compile en Lean 4. Voici le code final :

```lean
theorem add_zero_comm (n m : Nat) : n + m + 0 = m + n := by
  simpa [Nat.add_assoc] 

### 4.3. DEMO_3 : Reecritures Multiples

**Objectif** : Tester chaine de reecritures avec feedback

**Theoreme** : `theorem quad_comm (a b c d : Nat) : (a + b) + (c + d) = (d + c) + (b + a)`

**Attentes** :
- **Iterations** : 10-15 (4 reecritures successives)
- **Agents impliques** : SearchAgent → TacticAgent → VerifierAgent → CriticAgent (×3-4)
- **Lemmes Mathlib attendus** : `Nat.add_comm`, `Nat.add_assoc`, `Nat.add_left_comm`
- **CriticAgent** : REQUIS (plusieurs ajustements necessaires)
- **CoordinatorAgent** : PROBABLE (si blocage, strategie AC normalization)
- **Temps** : 3-8 secondes

Cette demo teste l'**orchestration multi-agents avec feedback loops** :
- Etape 1 : `rw [Nat.add_comm c d]` pour avoir `(d + c)`
- Etape 2 : `rw [Nat.add_comm a b]` pour avoir `(b + a)`
- Etape 3 : Eventuellement `rw [Nat.add_comm]` global
- Ou : `simp only [Nat.add_comm, Nat.add_assoc, Nat.add_left_comm]` (AC normalization)

In [32]:

# Execute DEMO_3
demo = DEMOS[2]
print("\n" + "=" * 70)
print(f"DEMO 3/4: {demo['name']}")
print("=" * 70)
print(f"Theoreme: {demo['theorem']}")
print(f"Complexite: {demo['complexity']}")
print(f"Iterations attendues: {demo['expected_iterations']}")
print(f"Lemmes necessaires: {demo['expected_lemmas']}")
print("=" * 70)

result_3 = prove_with_multi_agents(
    theorem=demo["theorem"],
    max_iterations=20,
    verbose=True,
    use_simulation=not USE_LLM_MODE
)

print(f"\nResultat DEMO_3:")
print(f"  - Success: {result_3['success']}")
print(f"  - Iterations: {result_3['iterations']}")
print(f"  - Proof: {result_3['final_proof'][:100] if result_3['final_proof'] else 'None'}...")



DEMO 3/4: DEMO_3_MULTI_REWRITE
Theoreme: theorem quad_comm (a b c d : Nat) : (a + b) + (c + d) = (d + c) + (b + a)
Complexite: Intermediaire - chaine de reecritures
Iterations attendues: 10-15
Lemmes necessaires: 2
Crees 5 agents SK avec modele gpt-5.2
Mode: Semantic Kernel
Session MULTI-AGENTS (SK): Prouver: theorem quad_comm (a b c d : Nat) : (a + b) + (c + d) = (d + c) + (b + ...

[Tour 2] SearchAgent
  Response: {"recipient_name":"functions.state-add_discovered_lemma","parameters":{"name":"Nat.add_left_comm","statement":"Nat.add_left_comm (n m k : Nat) : n + (m + k) = m + (n + k)","namespace":"Nat","relevance...

[Tour 3] TacticAgent
  Response: Tactique proposée (forte chance de marcher) :

```lean
by
  simpa [Nat.add_assoc, Nat.add_comm, Nat.add_left_comm]
```

À faire vérifier par `VerifierAgent`.

[Tour 4] VerifierAgent
  Response: La preuve compile correctement (seulement un warning “simp au lieu de simpa”).

```lean
theorem quad_comm (a b c d : Nat) : (a + b) + (c + d) = (d 

### 4.4. DEMO_4 : Preuve Structuree

**Objectif** : Stresser le systeme avec decomposition algebrique

**Theoreme** : `theorem distrib_both (a b c : Nat) : (a + b) * c + a * c = a * c + a * c + b * c`

**Attentes** :
- **Iterations** : 15-20 (decomposition + reordonnancement)
- **Agents impliques** : Tous les 5 agents avec changement de strategie
- **Lemmes Mathlib attendus** : `Nat.right_distrib`, `Nat.add_assoc`, `Nat.add_comm`
- **Strategies** : EXPLORATION → REFINEMENT → VALIDATION
- **CriticAgent/CoordinatorAgent** : **REQUIS**
- **Temps** : 5-15 secondes

Cette demo doit **declencher le workflow complet** :

1. **SearchAgent** : Trouve `Nat.right_distrib : (n + m) * k = n * k + m * k`
2. **TacticAgent** : Applique `rw [Nat.right_distrib]` → `a * c + b * c + a * c`
3. **VerifierAgent** : But non ferme (`a*c + b*c + a*c ≠ a*c + a*c + b*c`)
4. **CriticAgent** : Analyse → besoin de reassociation
5. **CoordinatorAgent** : Strategie `ring` ou reassociation manuelle
6. **TacticAgent** : `ring` ou `simp [Nat.add_comm, Nat.add_assoc]`
7. **VerifierAgent** : SUCCES

**Note** : Si DEMO_4 se complete en <10 iterations, cela signifie que la simulation est trop optimiste.

In [33]:

# Execute DEMO_4
demo = DEMOS[3]
print("\n" + "=" * 70)
print(f"DEMO 4/4: {demo['name']}")
print("=" * 70)
print(f"Theoreme: {demo['theorem']}")
print(f"Complexite: {demo['complexity']}")
print(f"Iterations attendues: {demo['expected_iterations']}")
print(f"Lemmes necessaires: {demo['expected_lemmas']}")
print("=" * 70)

result_4 = prove_with_multi_agents(
    theorem=demo["theorem"],
    max_iterations=20,
    verbose=True,
    use_simulation=not USE_LLM_MODE
)

print(f"\nResultat DEMO_4:")
print(f"  - Success: {result_4['success']}")
print(f"  - Iterations: {result_4['iterations']}")
print(f"  - Proof: {result_4['final_proof'][:100] if result_4['final_proof'] else 'None'}...")



DEMO 4/4: DEMO_4_STRUCTURED
Theoreme: theorem distrib_both (a b c : Nat) : (a + b) * c + a * c = a * c + a * c + b * c
Complexite: Avancee - decomposition + recombinaison
Iterations attendues: 15-20
Lemmes necessaires: 3
Crees 5 agents SK avec modele gpt-5.2
Mode: Semantic Kernel
Session MULTI-AGENTS (SK): Prouver: theorem distrib_both (a b c : Nat) : (a + b) * c + a * c = a * c + a * ...

[Tour 2] SearchAgent
  Response: 







































































































































































































...

[Tour 3] TacticAgent
  Response: Essaye cette preuve (développement puis normalisation par `simp`) :

```lean
theorem distrib_both (a b c : Nat) :
    (a + b) * c + a * c = a * c + a * c + b * c := by
  simp [Nat.right_distrib, Nat.a...

[LOG] Preuve trouvee!

Session terminee apres 3 iterations.

Resultat DEMO_4:
  - Success: True
  - Iterations: 3
  - Proof: theor

### 4.5. Analyse Comparative des Resultats

**Objectif** : Comparer les resultats observes avec les attentes.

#### Pourquoi les resultats sont-ils plus rapides que prevu ?

Les demos se terminent en 3-4 iterations au lieu de 10-20 car :

1. **Mathlib contient les lemmes exacts** : `Nat.add_right_cancel`, `Nat.mul_add`, `List.length_append`
2. **SearchAgent trouve immediatement** le bon lemme (pas de recherche exploratoire)
3. **TacticAgent applique directement** `simpa using <lemme>` sans essayer d'autres approches
4. **CriticAgent/CoordinatorAgent jamais actives** car aucun echec a corriger

#### Implications pedagogiques

| Aspect | Simulation actuelle | Systeme reel (LLM) |
|--------|--------------------|--------------------|
| **Recherche** | Base indexee, O(1) | Embedding similarity, exploration |
| **Tactiques** | Pattern matching | Generation creative, essais multiples |
| **Verification** | Heuristique simple | Lean 4 reel, erreurs detaillees |
| **Iterations** | 3-4 (deterministe) | 10-20 (stochastique) |

#### Pour observer la vraie complexite

```python
# Option 1: Mode LLM (necessite API key)
USE_LLM_MODE = True  # Active les vraies generations

# Option 2: Theoremes sans lemme direct
theorem_custom = "theorem custom (n m k : Nat) : (n + m) * k = n * k + m * k"
# Mathlib a Nat.add_mul mais pas dans notre base de simulation

# Option 3: Desactiver lemmes specifiques
# Modifier SIMULATION_LEMMAS pour exclure List.length_append
```

**Conclusion** : La simulation demontre l'*architecture* multi-agents, pas la *difficulte* reelle du theorem proving.

In [34]:

# =============================================================================
# Cellule 4.5 - Table de Comparaison des Resultats
# =============================================================================

# Collecter les resultats des 4 demos
try:
    all_results = [result_1, result_2, result_3, result_4]
except NameError:
    print("Erreur: Executez d'abord les cellules DEMO_1 a DEMO_4")
    all_results = None

if all_results:
    print("=" * 80)
    print("TABLEAU COMPARATIF DES RESULTATS")
    print("=" * 80)
    print()

    # Header
    print(f"{'Demo':<20} {'Succes':<8} {'Iter.':<8} {'Attendu':<10} {'Ecart':<10} {'Temps (s)':<10}")
    print("-" * 80)

    expected_iterations = ["1-2", "6-10", "10-15", "12-20"]

    for i, (result, expected) in enumerate(zip(all_results, expected_iterations), 1):
        demo_name = DEMOS[i-1]["name"]
        success = "OK" if result["success"] else "ECHEC"
        iterations = result["iterations"]
        time_s = result["total_time_s"]

        # Calculer ecart
        exp_min, exp_max = map(int, expected.split("-"))
        if iterations < exp_min:
            ecart = f"-{exp_min - iterations}"
        elif iterations > exp_max:
            ecart = f"+{iterations - exp_max}"
        else:
            ecart = "OK"

        print(f"{demo_name:<20} {success:<8} {iterations:<8} {expected:<10} {ecart:<10} {time_s:<10.2f}")

    print("-" * 80)

    # Statistiques globales
    total_success = sum(1 for r in all_results if r["success"])
    total_iterations = sum(r["iterations"] for r in all_results)
    total_time = sum(r["total_time_s"] for r in all_results)

    print(f"\nResume: {total_success}/4 succes, {total_iterations} iterations totales, {total_time:.2f}s")

    # Analyse
    print("\n" + "=" * 80)
    print("ANALYSE")
    print("=" * 80)

    if all(r["iterations"] <= 5 for r in all_results):
        print("\n[OBSERVATION] Toutes les demos terminent en <=5 iterations.")
        print("[CAUSE] La simulation trouve les lemmes Mathlib directement.")
        print("[IMPLICATION] CriticAgent et CoordinatorAgent jamais actives.")
        print("\n[SOLUTION] Pour tester l'orchestration complete:")
        print("  1. Utiliser USE_LLM_MODE = True (appels OpenAI reels)")
        print("  2. Ou modifier SIMULATION_LEMMAS pour exclure les lemmes directs")
    else:
        print("\n[OBSERVATION] Certaines demos necessitent plus d'iterations.")
        print("[BON SIGNE] L'orchestration complete peut etre observee.")


TABLEAU COMPARATIF DES RESULTATS

Demo                 Succes   Iter.    Attendu    Ecart      Temps (s) 
--------------------------------------------------------------------------------
DEMO_1_TRIVIAL       OK       3        1-2        +1         19.31     
DEMO_2_COMPOSITION   OK       4        6-10       -2         359.80    
DEMO_3_MULTI_REWRITE OK       4        10-15      -6         24.70     
DEMO_4_STRUCTURED    OK       3        12-20      -9         39.24     
--------------------------------------------------------------------------------

Resume: 4/4 succes, 14 iterations totales, 443.05s

ANALYSE

[OBSERVATION] Toutes les demos terminent en <=5 iterations.
[CAUSE] La simulation trouve les lemmes Mathlib directement.
[IMPLICATION] CriticAgent et CoordinatorAgent jamais actives.

[SOLUTION] Pour tester l'orchestration complete:
  1. Utiliser USE_LLM_MODE = True (appels OpenAI reels)
  2. Ou modifier SIMULATION_LEMMAS pour exclure les lemmes directs


## 5. Conclusion et Points Cles

### Ce que nous avons appris

#### 1. Architecture Multi-Agents pour Theorem Proving

| Composant | Role | Implementation SK |
|-----------|------|-------------------|
| **ProofState** | Etat partage synchronise | `@dataclass` + plugins |
| **Plugins** | Fonctionnalites specialisees | `@kernel_function` |
| **Agents** | Roles specialises (Search, Tactic, Verify...) | `ChatCompletionAgent` |
| **Orchestration** | Delegation dynamique | `AgentGroupChat` + strategies |

#### 2. Semantic Kernel vs Implementation Ad-Hoc (Lean-8)

| Aspect | Lean-8 (Ad-Hoc) | Lean-9 (Semantic Kernel) |
|--------|-----------------|--------------------------|
| **Etat** | Variables globales | `ProofState` classe |
| **Agents** | Fonctions Python | `ChatCompletionAgent` |
| **Communication** | Appels directs | Message passing |
| **Extensibilite** | Modifier le code | Ajouter plugins |
| **LLM** | OpenAI direct | Abstraction SK |

#### 3. Patterns Replicables

1. **StateManager Pattern** : Un objet central pour l'etat partage
2. **Plugin Pattern** : Fonctions decorees pour l'injection de dependances
3. **Delegation Pattern** : Chaque agent designe le suivant
4. **Termination Pattern** : Criteres multiples (succes, timeout, max_iter)

### Limitations et Perspectives

#### Limitations actuelles

- **Simulation trop parfaite** : Trouve les lemmes directs immediatement
- **Pas de vrai Lean** : Verification heuristique, pas de lean4 reel
- **Base de lemmes limitee** : ~50 lemmes vs 100k+ dans Mathlib
- **Pas de backtracking** : Premiere tactique qui marche = solution

#### Prochaines etapes (Lean-10 LeanDojo)

1. **Integration LeanDojo** : Interaction programmatique avec Lean 4
2. **Tracing Mathlib** : Extraction des 100k+ lemmes
3. **Verification reelle** : Feedback Lean vs heuristique
4. **Benchmarks** : MiniF2F, ProofNet, LeanBench

### Resume Final

Ce notebook a demontre comment construire un systeme multi-agents pour le theorem proving avec Semantic Kernel. Les patterns (StateManager, Plugin, Delegation) sont replicables pour d'autres domaines.

**Key Takeaways** :
- L'architecture compte plus que les resultats de simulation
- Semantic Kernel simplifie l'orchestration multi-agents
- Les vrais defis apparaissent avec des theoremes sans lemmes directs
- LeanDojo (Notebook 10) permettra la verification reelle

---

### Navigation

| Precedent | Index | Suivant |
|-----------|-------|---------|
| [Lean-8 (Agents Ad-Hoc)](Lean-8-Agentic-Proving.ipynb) | [Lean-1 (Setup)](Lean-1-Setup.ipynb) | [Lean-10 (LeanDojo)](Lean-10-LeanDojo.ipynb) |

---

*Notebook complete. Duree estimee: 45-55 minutes.*